## data import

In [2]:
import pkg_resources
import pip
installedPackages = {pkg.key for pkg in pkg_resources.working_set}
required = {'researchpy', 'missingno', 'folium', 'pydotplus','bokeh','imblearn', 'catboost', 'openpyxl'}
missing = required - installedPackages
if missing:
    !pip install researchpy
    !pip install missingno
    !pip install folium
    !pip install pydotplus
    !pip install bokeh
    !pip install imblearn
    !pip install catboost
    !pip install openpyxl
    #!pip install xgboost
    #!pip install graphviz

In [3]:
#Disable the warnings
import warnings
warnings.filterwarnings('ignore')

In [4]:
import timeit
start_time = timeit.default_timer()  #timestamp to calculate total runtime

import pandas as pd
import numpy as np

import researchpy as rp
import missingno as msno
import itertools
import scipy.stats as ss

import seaborn as sns
import matplotlib.pyplot as plt
import folium
from folium import plugins
import graphviz

from sklearn import tree
from sklearn import feature_selection
from sklearn import preprocessing

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

from collections import Counter

from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

from sklearn.utils import resample
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, accuracy_score, \
precision_score, recall_score, roc_auc_score, f1_score, precision_recall_curve, auc 

%matplotlib inline
pd.set_option('display.max_columns', None)  # prevent column output trancation
sns.set()  # change plot styling from Matlab's 90s feel to today's Seaborn.

In [5]:
# File Directories
path_crashes = '../crashes.sample2020.csv'
path_vehicles = '../vehicles.sample.csv'
path_people = '../people.sample.csv'

# Import samples
crashes = pd.read_csv(path_crashes, parse_dates=["CRASH_DATE", "CRASH_DATE_EST_I", "DATE_POLICE_NOTIFIED"],
                      low_memory=False, dtype=object)
vehicles = pd.read_csv(path_vehicles, parse_dates=["CRASH_DATE"], low_memory=False, dtype=object)
people = pd.read_csv(path_people, parse_dates=["CRASH_DATE"], low_memory=False, dtype=object)

In [6]:
# Joining datasets
non_passengers=people[people.PERSON_ID.str.contains('^O')]

vehicles_with_people=vehicles.merge(non_passengers,how='left',on=['CRASH_RECORD_ID','RD_NO','CRASH_DATE','VEHICLE_ID'])

data=crashes.merge(vehicles_with_people,how='inner',on=['CRASH_RECORD_ID','RD_NO','CRASH_DATE'])

# Feature Selection
filter_list=["AGE","LANE_CNT","AIRBAG_DEPLOYED","PRIM_CONTRIBUTORY_CAUSE","POSTED_SPEED_LIMIT","NUM_UNITS","TRAFFICWAY_TYPE",  
             "SEC_CONTRIBUTORY_CAUSE","FIRST_CRASH_TYPE","MOST_SEVERE_INJURY","LIGHTING_CONDITION","SEX","CRASH_DATE",
             "CRASH_HOUR","VEHICLE_YEAR"]

# Data that will be used in predictions
modeling_data=data[filter_list]

In [7]:
def preprocessor(dataframe):
    '''Preprocesses df and returns X and y ready for modeling (after imputation of numericals!)'''
    df = dataframe.copy()
    
    # Prepare data for missing value imputation
    df.loc[df["LIGHTING_CONDITION"] == "UNKNOWN", "LIGHTING_CONDITION"] = np.nan
    df.loc[df["TRAFFICWAY_TYPE"] == "UNKNOWN","TRAFFICWAY_TYPE"] = np.nan
    df.loc[df["AIRBAG_DEPLOYED"] == "DEPLOYMENT UNKNOWN","AIRBAG_DEPLOYED"] = np.nan
    df.fillna({'LIGHTING_CONDITION': 'DAYLIGHT', 'TRAFFICWAY_TYPE': 'NOT DIVIDED',
               'SEX': 'UNABLE TO DETERMINE', 'AIRBAG_DEPLOYED': 'UNABLE TO DETERMINE'}, inplace=True)
    
    df.loc[df["TRAFFIC_CONTROL_DEVICE"]=="UNKNOWN","TRAFFIC_CONTROL_DEVICE"]=np.nan
    df.loc[df["DEVICE_CONDITION"]=="UNKNOWN","DEVICE_CONDITION"]=np.nan
    df.loc[df["WEATHER_CONDITION"]=="UNKNOWN","WEATHER_CONDITION"]=np.nan
    df.loc[df["ROADWAY_SURFACE_COND"]=="UNKNOWN","ROADWAY_SURFACE_COND"]=np.nan
    df.loc[df["ROAD_DEFECT"]=="UNKNOWN","ROAD_DEFECT"]=np.nan

    df.loc[df["VEHICLE_DEFECT"]=="UNKNOWN","VEHICLE_DEFECT"]=np.nan
    df.loc[df["VEHICLE_TYPE"]=="UNKNOWN/NA","VEHICLE_TYPE"]=np.nan
    df.loc[df["TRAVEL_DIRECTION"]=="UNKNOWN","TRAVEL_DIRECTION"]=np.nan
    df.loc[df["MANEUVER"]=="UNKNOWN/NA","MANEUVER"]=np.nan

    df.loc[df["SAFETY_EQUIPMENT"]=="USAGE UNKNOWN","SAFETY_EQUIPMENT"]=np.nan
    df.loc[df["AIRBAG_DEPLOYED"]=="DEPLOYMENT UNKNOWN","AIRBAG_DEPLOYED"]=np.nan
    df.loc[df["EJECTION"]=="UNKNOWN","EJECTION"]=np.nan
    df.loc[df["DRIVER_ACTION"]=="UNKNOWN","DRIVER_ACTION"]=np.nan
    df.loc[df["DRIVER_VISION"]=="UNKNOWN","DRIVER_VISION"]=np.nan
    df.loc[df["PHYSICAL_CONDITION"]=="UNKNOWN","PHYSICAL_CONDITION"]=np.nan
    df.loc[df["PEDPEDAL_ACTION"]=="UNKNOWN/NA","PEDPEDAL_ACTION"]=np.nan
    
    # Remove rows missing most severe injury results
    drop_rows = ['MOST_SEVERE_INJURY']
    df.dropna(how ='any', subset = drop_rows, inplace = True)
    
    # Handle numerical features
    df['VEHICLE_YEAR'] = pd.to_numeric(df['VEHICLE_YEAR'])
    df['NUM_UNITS'] = pd.to_numeric(df['NUM_UNITS'])
    df["POSTED_SPEED_LIMIT"] = pd.to_numeric(df["POSTED_SPEED_LIMIT"])
    df["AGE"] = pd.to_numeric(df["AGE"])
    
    df['LANE_CNT'] = pd.to_numeric(df['LANE_CNT'])    
    df['LANE_CNT'].fillna(2, inplace=True)
    df.loc[(df['LANE_CNT'] > 6),'LANE_CNT'] = 6
    
    df.fillna({
    'TRAFFIC_CONTROL_DEVICE': 'NO CONTROLS',
    'DEVICE_CONDITION': 'NO CONTROLS',
    'WEATHER_CONDITION': 'CLEAR',
    'LIGHTING_CONDITION': 'DAYLIGHT',
    'TRAFFICWAY_TYPE': 'NOT DIVIDED',
    'ROADWAY_SURFACE_COND': 'NO DEFECTS',
    'ROAD_DEFECT': 'CLEAR',
    'INTERSECTION_RELATED_I': 'N',
    
    'NOT_RIGHT_OF_WAY_I': 'N',
    'HIT_AND_RUN_I': 'N',
    'DOORING_I': 'N',
    'WORK_ZONE_I': 'N',
    'NUM_PASSENGERS': 0,
    'LIC_PLATE_STATE': 'IL',
    'VEHICLE_DEFECT': 'UNABLE TO DETERMINE',
    'VEHICLE_TYPE': 'OTHER',
    
    'VEHICLE_USE': 'OTHER',
    'TRAVEL_DIRECTION': 'N',
    'MANEUVER': 'OTHER',
    'OCCUPANT_CNT': 0,
    'EXCEED_SPEED_LIMIT_I': 'N',
    'FIRST_CONTACT_POINT': 'OTHER',
    'PERSON_TYPE': 'UNABLE TO DETERMINE',
    'CITY': 'OTHER',
    
    'SEX': 'UNABLE TO DETERMINE',
    'AIRBAG_DEPLOYED': 'UNABLE TO DETERMINE',
    'EJECTION': 'UNABLE TO DETERMINE',
    'DRIVER_ACTION': 'OTHER',
    'DRIVER_VISION': 'OTHER',
    'PHYSICAL_CONDITION': 'UNABLE TO DETERMINE',
    'PEDPEDAL_ACTION': 'UNABLE TO DETERMINE',
    'PEDPEDAL_VISIBILITY': 'UNABLE TO DETERMINE',
    
    'CELL_PHONE_USE': 'UNABLE TO DETERMINE',
    'SAFETY_EQUIPMENT': 'UNABLE TO DETERMINE',
    'BAC_RESULT VALUE': 0,
    'VEHICLE_YEAR':0
}, inplace=True)
    
    # drop columns
    drop_columns = ["CRASH_DATE_EST_I", "REPORT_TYPE", "CRASH_DATE_EST_I", "REPORT_TYPE", 
                      "DATE_POLICE_NOTIFIED","BEAT_OF_OCCURRENCE", "PHOTOS_TAKEN_I", "STATEMENTS_TAKEN_I", 
                      "WORK_ZONE_TYPE", "WORKERS_PRESENT_I","INJURIES_NO_INDICATION", "INJURIES_UNKNOWN",
                      "VEHICLE_ID", "CRASH_RECORD_ID", "STATE", "PERSON_ID", "DRIVERS_LICENSE_CLASS", "INJURY_CLASSIFICATION", "HOSPITAL", "EMS_AGENCY", 
                      "EMS_RUN_NO", "PEDPEDAL_LOCATION", "LOCATION","DAMAGE","CRASH_TYPE","MODEL",
                      "BAC_RESULT", "CRASH_UNIT_ID", "RD_NO", "UNIT_NO", "UNIT_TYPE", "VEHICLE_ID", 
                      "CMRC_VEH_I", "MAKE", "TOWED_I", "FIRE_I", "TOWED_BY", "STREET_NO","STREET_NAME",
                      "TOWED_TO", "AREA_00_I", "AREA_01_I", "AREA_02_I", "AREA_03_I", "AREA_04_I", 
                      "AREA_05_I", "AREA_06_I", "AREA_07_I", "AREA_08_I", "AREA_09_I", "AREA_10_I", 
                      "AREA_11_I", "AREA_12_I", "AREA_99_I", "CMV_ID", "USDOT_NO", "CCMC_NO", 
                      "ILCC_NO", "COMMERCIAL_SRC", "GVWR", "CARRIER_NAME", "CARRIER_STATE", "CARRIER_CITY",
                      "HAZMAT_PLACARDS_I", "HAZMAT_NAME", "UN_NO", "HAZMAT_PRESENT_I", "HAZMAT_REPORT_I",
                      "HAZMAT_REPORT_NO", "MCS_REPORT_I", "MCS_REPORT_NO", "HAZMAT_VIO_CAUSE_CRASH_I", 
                      "MCS_VIO_CAUSE_CRASH_I", "IDOT_PERMIT_NO", "WIDE_LOAD_I", "TRAILER1_WIDTH", 
                      "TRAILER2_WIDTH", "TRAILER1_LENGTH", "TRAILER2_LENGTH", "TOTAL_VEHICLE_LENGTH",
                      "AXLE_CNT", "VEHICLE_CONFIG", "CARGO_BODY_TYPE", "LOAD_TYPE", "HAZMAT_OUT_OF_SERVICE_I",
                      "INJURIES_FATAL","INJURIES_INCAPACITATING","INJURIES_NON_INCAPACITATING","INJURIES_REPORTED_NOT_EVIDENT",
                      "MCS_OUT_OF_SERVICE_I", "HAZMAT_CLASS", "ZIPCODE", "DRIVERS_LICENSE_STATE","SEAT_NO"]
    df = df.loc[:, ~df.columns.isin(drop_columns)]
    
    drop_rows2 = ['LATITUDE','MOST_SEVERE_INJURY']
    df.dropna(how='any', subset=drop_rows2, inplace = True)
    
    
    # Function definitions
    def injury(x): 
        if any(s in x for s in ["FATAL","NONINCAPACITATING INJURY","INCAPACITATING INJURY"]):
            return "INJURED"
        else:
            return "NOT INJURED"
    
    def airbag(x):
        if ("DEPLOY" in x) and ("UNKNOWN" not in x):
            if "NOT" in x:
                return "NOT DEPLOYED"
            else:
                return "DEPLOYED"
        else:
            return x

    def crash_hour(x):
        if  2 <= x < 8:
            return "Early_morning"
        elif 8 <= x < 12:
            return "Morning"
        elif 12 <= x < 18:
            return "Afternoon"
        else:
            return "Night"
  
    def traffic_way(x):
        if ("NOT" in x) or ("ONE-WAY" in x):
            return "NOT_DIVIDED"
        else:
            return "DIVIDED"
        
#     def age_group(x):
#         if  0.0 <= x <= 18.0:
#             return "below 18"
#         elif 18.0 < x <= 30.0:
#             return "between 19 and 30"
#         elif 30.0< x <= 40.0:
#             return "between 31 and 40"
#         elif 40.0< x <= 50.0:
#             return "between 41 and 50"
#         elif 50.0< x <= 60.0:
#             return "between 51 and 60"
#         else:
#             return "Over 60"
        
    def age_group(x):
        if  0.0 <= x <= 18.0:
            return "Under 18"
        elif 18.0 < x <= 30.0:
            return "Between 19 and 30"
        elif 30.0 < x <= 60.0:
            return "Middle Age 30 to 60"
        else:
            return "Older than 60"
    
    # Feature Engineering
    df["INJURY"] = df["MOST_SEVERE_INJURY"].apply(lambda x: injury(x))
    df["AIRBAG_DEPLOYED"] = df["AIRBAG_DEPLOYED"].apply(lambda x: airbag(x))
    df["CRASH_HOUR"] = df["CRASH_HOUR"].apply(lambda x: crash_hour(int(x)))
    df["TRAFFICWAY_TYPE"] = df["TRAFFICWAY_TYPE"].apply(lambda x: traffic_way(x))
    df["AGE"] = df["AGE"].apply(lambda x: age_group(x))
    
    df["VEHICLE_AGE"] = df["CRASH_DATE"].dt.year-df["VEHICLE_YEAR"]
    df.loc[df["VEHICLE_AGE"] < 0, "VEHICLE_AGE"] = 0
    df.drop(["VEHICLE_YEAR", "CRASH_DATE", "MOST_SEVERE_INJURY"], axis=1, inplace=True)
    
#     # Splitting df into X and y
#     y = df["INJURY"]
#     X = df.drop(["INJURY"], axis=1)
    
#     # Binarize y
#     from sklearn.preprocessing import label_binarize
#     y = preprocessing.label_binarize(y, classes=['NOT INJURED', 'INJURED'])
    
#     # One-Hot Encoding
#     X = pd.get_dummies(X, columns = X.select_dtypes(['object']).columns)
#     dummies_to_drop = X.columns[X.columns.str.contains("UNABLE|UNKNOWN|NOT APPLICABLE|OTHER")]
#     X = X.loc[:, ~X.columns.isin(dummies_to_drop)]
      
    return(df)

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68503 entries, 0 to 68502
Columns: 144 entries, CRASH_RECORD_ID to CELL_PHONE_USE
dtypes: datetime64[ns](2), object(142)
memory usage: 75.8+ MB


In [9]:
data.head()

,CRASH_RECORD_ID,RD_NO,CRASH_DATE_EST_I,CRASH_DATE,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,TRAFFICWAY_TYPE,LANE_CNT,ALIGNMENT,ROADWAY_SURFACE_COND,ROAD_DEFECT,REPORT_TYPE,CRASH_TYPE,INTERSECTION_RELATED_I,NOT_RIGHT_OF_WAY_I,HIT_AND_RUN_I,DAMAGE,DATE_POLICE_NOTIFIED,PRIM_CONTRIBUTORY_CAUSE,SEC_CONTRIBUTORY_CAUSE,STREET_NO,STREET_DIRECTION,STREET_NAME,BEAT_OF_OCCURRENCE,PHOTOS_TAKEN_I,STATEMENTS_TAKEN_I,DOORING_I,WORK_ZONE_I,WORK_ZONE_TYPE,WORKERS_PRESENT_I,NUM_UNITS,MOST_SEVERE_INJURY,INJURIES_TOTAL,INJURIES_FATAL,INJURIES_INCAPACITATING,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,LOCATION,CRASH_UNIT_ID,UNIT_NO,UNIT_TYPE,NUM_PASSENGERS,VEHICLE_ID,CMRC_VEH_I,MAKE,MODEL,LIC_PLATE_STATE,VEHICLE_YEAR,VEHICLE_DEFECT,VEHICLE_TYPE,VEHICLE_USE,TRAVEL_DIRECTION,MANEUVER,TOWED_I,FIRE_I,OCCUPANT_CNT,EXCEED_SPEED_LIMIT_I,TOWED_BY,TOWED_TO,AREA_00_I,AREA_01_I,AREA_02_I,AREA_03_I,AREA_04_I,AREA_05_I,AREA_06_I,AREA_07_I,AREA_08_I,AREA_09_I,AREA_10_I,AREA_11_I,AREA_12_I,AREA_99_I,FIRST_CONTACT_POINT,CMV_ID,USDOT_NO,CCMC_NO,ILCC_NO,COMMERCIAL_SRC,GVWR,CARRIER_NAME,CARRIER_STATE,CARRIER_CITY,HAZMAT_PLACARDS_I,HAZMAT_NAME,UN_NO,HAZMAT_PRESENT_I,HAZMAT_REPORT_I,HAZMAT_REPORT_NO,MCS_REPORT_I,MCS_REPORT_NO,HAZMAT_VIO_CAUSE_CRASH_I,MCS_VIO_CAUSE_CRASH_I,IDOT_PERMIT_NO,WIDE_LOAD_I,TRAILER1_WIDTH,TRAILER2_WIDTH,TRAILER1_LENGTH,TRAILER2_LENGTH,TOTAL_VEHICLE_LENGTH,AXLE_CNT,VEHICLE_CONFIG,CARGO_BODY_TYPE,LOAD_TYPE,HAZMAT_OUT_OF_SERVICE_I,MCS_OUT_OF_SERVICE_I,HAZMAT_CLASS,PERSON_ID,PERSON_TYPE,SEAT_NO,CITY,STATE,ZIPCODE,SEX,AGE,DRIVERS_LICENSE_STATE,DRIVERS_LICENSE_CLASS,SAFETY_EQUIPMENT,AIRBAG_DEPLOYED,EJECTION,INJURY_CLASSIFICATION,HOSPITAL,EMS_AGENCY,EMS_RUN_NO,DRIVER_ACTION,DRIVER_VISION,PHYSICAL_CONDITION,PEDPEDAL_ACTION,PEDPEDAL_VISIBILITY,PEDPEDAL_LOCATION,BAC_RESULT,BAC_RESULT VALUE,CELL_PHONE_USE
0,0211e1f766f3940dfa87375661d25b716655e908c320cc...,JC301403,NaN,2019-06-11 08:40:00,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,REAR END,DIVIDED - W/MEDIAN BARRIER,NaN,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NOT ON SCENE (DESK REPORT),NO INJURY / DRIVE AWAY,Y,NaN,NaN,"$501 - $1,500",2019-06-11 09:05:00,UNABLE TO DETERMINE,NOT APPLICABLE,50,E,GARFIELD BLVD,225,NaN,NaN,NaN,NaN,NaN,NaN,2,NO INDICATION OF INJURY,0,0,0,0,0,3,0,8,3,6,41.794778764,-87.623828038,POINT (-87.623828038036 41.794778764028),667550,1,DRIVER,NaN,635502,NaN,CHEVROLET,EQUINOX,IL,2017,UNKNOWN,SPORT UTILITY VEHICLE (SUV),UNKNOWN/NA,W,STRAIGHT AHEAD,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,OTHER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,O667550,DRIVER,NaN,HOMEWOOD,IL,60430,F,56,IL,D,USAGE UNKNOWN,DID NOT DEPLOY,NONE,NO INDICATION OF INJURY,NaN,NaN,NaN,UNKNOWN,UNKNOWN,NORMAL,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
1,0211e1f766f3940dfa87375661d25b716655e908c320cc...,JC301403,NaN,2019-06-11 08:40:00,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,DAYLIGHT,REAR END,DIVIDED - W/MEDIAN BARRIER,NaN,STRAIGHT AND LEVEL,DRY,NO DEFECTS,NOT ON SCENE (DESK REPORT),NO INJURY / DRIVE AWAY,Y,NaN,NaN,"$501 - $1,500",2019-06-11 09:05:00,UNABLE TO DETERMINE,NOT APPLICABLE,50,E,GARFIELD BLVD,225,NaN,NaN,NaN,NaN,NaN,NaN,2,NO INDICATION OF INJURY,0,0,0,0,0,3,0,8,3,6,41.794778764,-87.623828038,POINT (-87.623828038036 41.794778764028),667551,2,DRIVER,1,635496,NaN,CHEVROLET,SILVERADO,IL,2016,NONE,PICKUP,PERSONAL,W,STARTING IN TRAFFIC,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,REAR-LEFT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,O667551,DRIVER,NaN,PALOS HILLS,IL,60465,M,51,IL,D,SAFETY BELT USED,DID NOT DEPLOY,NONE,NO INDICATION OF INJURY,NaN,NaN,NaN,NONE,NOT OBSCURED,NORMAL,NaN,NaN,NaN,TEST 

In [32]:
processed_data = preprocessor(data)

In [110]:
processed_data['SEX'] = processed_data['SEX'].replace(['UNABLE TO DETERMINE', "X"], np.nan)
processed_data['SAFETY_EQUIPMENT'] = processed_data['SAFETY_EQUIPMENT'].replace(['UNABLE TO DETERMINE'], np.nan)
processed_data['SEC_CONTRIBUTORY_CAUSE'] = processed_data['SEC_CONTRIBUTORY_CAUSE'].replace(['UNABLE TO DETERMINE', 'NOT APPLICABLE'], np.nan)
processed_data['PHYSICAL_CONDITION'] = processed_data['PHYSICAL_CONDITION'].replace(['UNABLE TO DETERMINE'], np.nan)

processed_data['AGE_SEX_GROUP']= processed_data['AGE'] + "_" + processed_data['SEX']
processed_data['AGE_SEX_GROUP']

processed_data.groupby(['AGE_SEX_GROUP']).size()

AGE_SEX_GROUP
Between 19 and 30_F      4035
Between 19 and 30_M      5125
Middle Age 30 to 60_F    6100
Middle Age 30 to 60_M    9934
Older than 60_F          2876
Older than 60_M          5639
Under 18_F                488
Under 18_M                696
dtype: int64

In [111]:
processed_data.groupby(['PRIM_CONTRIBUTORY_CAUSE']).size()

PRIM_CONTRIBUTORY_CAUSE
ANIMAL                                                                                57
BICYCLE ADVANCING LEGALLY ON RED LIGHT                                                10
CELL PHONE USE OTHER THAN TEXTING                                                    115
DISREGARDING OTHER TRAFFIC SIGNS                                                     142
DISREGARDING ROAD MARKINGS                                                            83
DISREGARDING STOP SIGN                                                               861
DISREGARDING TRAFFIC SIGNALS                                                        1420
DISREGARDING YIELD SIGN                                                               20
DISTRACTION - FROM INSIDE VEHICLE                                                    454
DISTRACTION - FROM OUTSIDE VEHICLE                                                   335
DISTRACTION - OTHER ELECTRONIC DEVICE (NAVIGATION DEVICE, DVD PLAYER, ETC.)           

In [91]:
processed_data = processed_data[~processed_data['PRIM_CONTRIBUTORY_CAUSE'].str.contains("NOT APPLICABLE")]
processed_data = processed_data[~processed_data['PRIM_CONTRIBUTORY_CAUSE'].str.contains("UNABLE TO DETERMINE")]


In [92]:
processed_data.groupby(['PRIM_CONTRIBUTORY_CAUSE']).size()

PRIM_CONTRIBUTORY_CAUSE
ANIMAL                                                                                57
BICYCLE ADVANCING LEGALLY ON RED LIGHT                                                10
CELL PHONE USE OTHER THAN TEXTING                                                    115
DISREGARDING OTHER TRAFFIC SIGNS                                                     142
DISREGARDING ROAD MARKINGS                                                            83
DISREGARDING STOP SIGN                                                               861
DISREGARDING TRAFFIC SIGNALS                                                        1420
DISREGARDING YIELD SIGN                                                               20
DISTRACTION - FROM INSIDE VEHICLE                                                    454
DISTRACTION - FROM OUTSIDE VEHICLE                                                   335
DISTRACTION - OTHER ELECTRONIC DEVICE (NAVIGATION DEVICE, DVD PLAYER, ETC.)           

In [93]:
import researchpy as rp
table3, results3 = rp.crosstab(processed_data['AGE_SEX_GROUP'], processed_data['PRIM_CONTRIBUTORY_CAUSE'], 
                             prop='col', test='chi-square')

display(table3)
results3

PRIM_CONTRIBUTORY_CAUSE  \
PRIM_CONTRIBUTORY_CAUSE                  ANIMAL   
AGE_SEX_GROUP                                     
Between 19 and 30_F                       13.95   
Between 19 and 30_M                        9.30   
Middle Age 30 to 60_F                     30.23   
Middle Age 30 to 60_M                     25.58   
Older than 60_F                            9.30   
Older than 60_M                            2.33   
Under 18_F                                 0.00   
Under 18_M                                 9.30   
All                                      100.00   

                                                                \
PRIM_CONTRIBUTORY_CAUSE BICYCLE ADVANCING LEGALLY ON RED LIGHT   
AGE_SEX_GROUP                                                    
Between 19 and 30_F                                       12.5   
Between 19 and 30_M                                       12.5   
Middle Age 30 to 60_F                                     37.5   
Middle Age 30 to 60_M                                     12.5   
Older than 60_F                                           12.5   
Older than 60_M                                           12.5   
Under 18_F                                                 0.0   
Under 18_M                                                 0.0   
All                                                      100.0   

                                                           \
PRIM_CONTRIBUTORY_CAUSE CELL PHONE USE OTHER THAN TEXTING   
AGE_SEX_GROUP                                               
Between 19 and 30_F                                 20.24   
Between 19 and 30_M                                 17.86   
Middle Age 30 to 60_F                               11.90   
Middle Age 30 to 60_M                               30.95   
Older than 60_F                                      7.14   
Older than 60_M                                     10.71   
Under 18_F                                           1.19   
Under 18_M                                           0.00   
All                                                100.00   

                                                          \
PRIM_CONTRIBUTORY_CAUSE DISREGARDING OTHER TRAFFIC SIGNS   
AGE_SEX_GROUP                                              
Between 19 and 30_F                                 9.85   
Between 19 and 30_M                                16.67   
Middle Age 30 to 60_F                              18.18   
Middle Age 30 to 60_M                              39.39   
Older than 60_F                                     3.79   
Older than 60_M                                     9.09   
Under 18_F                                          2.27   
Under 18_M                                          0.76   
All                                               100.00   

                                                                           \
PRIM_CONTRIBUTORY_CAUSE DISREGARDING ROAD MARKINGS DISREGARDING STOP SIGN   
AGE_SEX_GROUP                                                               
Between 19 and 30_F                           4.00                  13.93   
Between 19 and 30_M                          12.00                  14.72   
Middle Age 30 to 60_F                        18.67                  19.50   
Middle Age 30 to 60_M                        34.67                  23.74   
Older than 60_F                               6.67                   7.82   
Older than 60_M                              22.67                  16.84   
Under 18_F                                    0.00                   1.46   
Under 18_M                                    1.33                   1.99   
All                                         100.00                 100.00   

                                                                              \
PRIM_CONTRIBUTORY_CAUSE DISREGARDING TRAFFIC SIGNALS DISREGARDING YIELD SIGN   
AGE_SEX_GROUP                                                                  

,Chi-square test,results
0,Pearson Chi-square ( 252.0) =,806.7326
1,p-value =,0.0000
2,Cramer's V =,0.0575


In [94]:
table3_2, results3_2 = rp.crosstab(processed_data['PRIM_CONTRIBUTORY_CAUSE'], processed_data['AGE_SEX_GROUP'], 
                             prop='col', test='chi-square')
display(table3_2)
results3_2

AGE_SEX_GROUP  \
AGE_SEX_GROUP                                      Between 19 and 30_F   
PRIM_CONTRIBUTORY_CAUSE                                                  
ANIMAL                                                            0.15   
BICYCLE ADVANCING LEGALLY ON RED LIGHT                            0.02   
CELL PHONE USE OTHER THAN TEXTING                                 0.42   
DISREGARDING OTHER TRAFFIC SIGNS                                  0.32   
DISREGARDING ROAD MARKINGS                                        0.07   
DISREGARDING STOP SIGN                                            2.60   
DISREGARDING TRAFFIC SIGNALS                                      3.67   
DISREGARDING YIELD SIGN                                           0.00   
DISTRACTION - FROM INSIDE VEHICLE                                 1.34   
DISTRACTION - FROM OUTSIDE VEHICLE                                0.87   
DISTRACTION - OTHER ELECTRONIC DEVICE (NAVIGATI...                0.12   
DRIVING ON WRONG SIDE/WRONG WAY                                   0.79   
DRIVING SKILLS/KNOWLEDGE/EXPERIENCE                               4.51   
EQUIPMENT - VEHICLE CONDITION                                     1.14   
EVASIVE ACTION DUE TO ANIMAL, OBJECT, NONMOTORIST                 0.17   
EXCEEDING AUTHORIZED SPEED LIMIT                                  0.37   
EXCEEDING SAFE SPEED FOR CONDITIONS                               0.55   
FAILING TO REDUCE SPEED TO AVOID CRASH                            8.35   
FAILING TO YIELD RIGHT-OF-WAY                                    23.57   
FOLLOWING TOO CLOSELY                                            19.23   
HAD BEEN DRINKING (USE WHEN ARREST IS NOT MADE)                   0.05   
IMPROPER BACKING                                                  4.68   
IMPROPER LANE USAGE                                               5.87   
IMPROPER OVERTAKING/PASSING                                       7.31   
IMPROPER TURNING/NO SIGNAL                                        6.29   
MOTORCYCLE ADVANCING LEGALLY ON RED LIGHT                         0.00   
OPERATING VEHICLE IN ERRATIC, RECKLESS, CARELES...                1.36   
PASSING STOPPED SCHOOL BUS                                        0.00   
PHYSICAL CONDITION OF DRIVER                                      0.62   
RELATED TO BUS STOP                                               0.02   
ROAD CONSTRUCTION/MAINTENANCE                                     0.25   
ROAD ENGINEERING/SURFACE/MARKING DEFECTS                          0.50   
TEXTING                                                           0.12   
TURNING RIGHT ON RED                                              0.07   
UNDER THE INFLUENCE OF ALCOHOL/DRUGS (USE WHEN ...                0.82   
VISION OBSCURED (SIGNS, TREE LIMBS, BUILDINGS, ...                1.07   
WEATHER                                                           2.68   
All                                                             100.00   

                                                                        \
AGE_SEX_GROUP                                      Between 19 and 30_M   
PRIM_CONTRIBUTORY_CAUSE                                                  
ANIMAL                                                            0.08   
BICYCLE ADVANCING LEGALLY ON RED LIGHT                            0.02   
CELL PHONE USE OTHER THAN TEXTING                                 0.29   
DISREGARDING OTHER TRAFFIC SIGNS                                  0.43   
DISREGARDING ROAD MARKINGS                                        0.18   
DISREGARDING STOP SIGN                                            2.17   
DISREGARDING TRAFFIC SIGNALS                                      4.59   
DISREGARDING YIELD SIGN                                           0.04   
DISTRACTION - FROM INSIDE VEHICLE                                 1.11   
DISTRACTION - FROM OUTSIDE VEHICLE                                0.88   
DISTRACTION - OTHER ELECTRONIC DEVICE (NAVIGATI...          

,Chi-square test,results
0,Pearson Chi-square ( 252.0) =,806.7326
1,p-value =,0.0000
2,Cramer's V =,0.0575


In [95]:
table3_2.to_excel(r'/Users/shellylin/repo_homlsl/Primary_chi_square2_modified.xlsx', header = True)
results3_2.to_excel(r'/Users/shellylin/repo_homlsl/Primary_chi_square_result2_modified.xlsx', header = True)


In [96]:
table4, results4 = rp.crosstab(processed_data['FIRST_CRASH_TYPE'], processed_data['AGE_SEX_GROUP'], 
                             prop='col', test='chi-square')

display(table4)
results4

AGE_SEX_GROUP                      \
AGE_SEX_GROUP                Between 19 and 30_F Between 19 and 30_M   
FIRST_CRASH_TYPE                                                       
ANGLE                                      16.41               15.88   
ANIMAL                                      0.05                0.04   
FIXED OBJECT                                2.06                3.34   
HEAD ON                                     1.19                1.01   
OTHER NONCOLLISION                          0.12                0.18   
OTHER OBJECT                                0.35                0.55   
OVERTURNED                                  0.05                0.04   
PARKED MOTOR VEHICLE                        5.30                6.22   
PEDALCYCLIST                                1.19                1.85   
PEDESTRIAN                                  2.68                1.93   
REAR END                                   29.96               28.78   
REAR TO FRONT                               0.77                0.78   
REAR TO REAR                                0.07                0.10   
REAR TO SIDE                                0.47                0.43   
SIDESWIPE OPPOSITE DIRECTION                1.39                1.39   
SIDESWIPE SAME DIRECTION                   15.69               15.26   
TRAIN                                       0.02                0.00   
TURNING                                    22.23               22.22   
All                                       100.00              100.00   

                                                                          \
AGE_SEX_GROUP                Middle Age 30 to 60_F Middle Age 30 to 60_M   
FIRST_CRASH_TYPE                                                           
ANGLE                                        15.46                 13.61   
ANIMAL                                        0.11                  0.04   
FIXED OBJECT                                  1.66                  2.73   
HEAD ON                                       0.70                  1.17   
OTHER NONCOLLISION                            0.15                  0.15   
OTHER OBJECT                                  0.28                  0.64   
OVERTURNED                                    0.03                  0.04   
PARKED MOTOR VEHICLE                          4.97                  6.37   
PEDALCYCLIST                                  1.20                  1.59   
PEDESTRIAN                                    2.48                  2.51   
REAR END                                     34.67                 30.46   
REAR TO FRONT                                 0.61                  0.90   
REAR TO REAR                                  0.18                  0.17   
REAR TO SIDE                                  0.48                  0.49   
SIDESWIPE OPPOSITE DIRECTION                  1.90                  1.54   
SIDESWIPE SAME DIRECTION                     16.07                 17.85   
TRAIN                                         0.00                  0.00   
TURNING                                      19.07                 19.74   
All                                         100.00                100.00   

                                                                         \
AGE_SEX_GROUP                Older than 60_F Older than 60_M Under 18_F   
FIRST_CRASH_TYPE                                                          
ANGLE                                  14.19           13.00      16.80   
ANIMAL                                  0.17            0.04       0.00   
FIXED OBJECT                            1.08            2.16       3.28   
HEAD ON                                 0.87            0.89       0.82   
OTHER NONCOLLISION                      0.07            0.09       0.00   
OTHER OBJECT                            0.38            0.64       0.61   
OVERTURNED                              0.03            0.02       0.00   
PARKED MOTOR VEHICLE       

,Chi-square test,results
0,Pearson Chi-square ( 119.0) =,745.2027
1,p-value =,0.0000
2,Cramer's V =,0.0552


In [97]:
# table4.to_excel(r'/Users/shellylin/repo_homlsl/First_crash_type_chi_square2.xlsx', header = True)
# results4.to_excel(r'/Users/shellylin/repo_homlsl/First_crash_type_chi_square_result.xlsx', header = True)

In [98]:
# table5.to_excel(r'/Users/shellylin/repo_homlsl/Lighting_condition_chi_square.xlsx', header = True)
# results5.to_excel(r'/Users/shellylin/repo_homlsl/Lighting_condition_chi_square_result.xlsx', header = True)

In [99]:
table5, results5 = rp.crosstab(processed_data['LIGHTING_CONDITION'], processed_data['AGE_SEX_GROUP'], 
                             prop='col', test='chi-square')

display(table5)
results5

AGE_SEX_GROUP                      \
AGE_SEX_GROUP          Between 19 and 30_F Between 19 and 30_M   
LIGHTING_CONDITION                                               
DARKNESS                              4.09                4.80   
DARKNESS, LIGHTED ROAD               23.30               28.94   
DAWN                                  1.66                1.60   
DAYLIGHT                             67.83               61.54   
DUSK                                  3.12                3.12   
All                                 100.00              100.00   

                                                                    \
AGE_SEX_GROUP          Middle Age 30 to 60_F Middle Age 30 to 60_M   
LIGHTING_CONDITION                                                   
DARKNESS                                4.03                  4.66   
DARKNESS, LIGHTED ROAD                 17.43                 21.85   
DAWN                                    1.69                  2.15   
DAYLIGHT                               73.74                 68.44   
DUSK                                    3.11                  2.89   
All                                   100.00                100.00   

                                                                              \
AGE_SEX_GROUP          Older than 60_F Older than 60_M Under 18_F Under 18_M   
LIGHTING_CONDITION                                                             
DARKNESS                          4.45            4.68       3.69       4.45   
DARKNESS, LIGHTED ROAD           14.57           20.68      19.06      25.29   
DAWN                              1.50            1.45       1.02       1.44   
DAYLIGHT                         76.50           70.08      73.98      64.37   
DUSK                              2.99            3.10       2.25       4.45   
All                             100.00          100.00     100.00     100.00   

                                
AGE_SEX_GROUP              All  
LIGHTING_CONDITION              
DARKNESS                  4.47  
DARKNESS, LIGHTED ROAD   21.53  
DAWN                      1.74  
DAYLIGHT                 69.21  
DUSK                      3.06  
All                     100.00

,Chi-square test,results
0,Pearson Chi-square ( 28.0) =,377.422
1,p-value =,0.000
2,Cramer's V =,0.052


In [100]:
table6, results6 = rp.crosstab(processed_data['SEC_CONTRIBUTORY_CAUSE'], processed_data['AGE_SEX_GROUP'], 
                             prop='col', test='chi-square')

display(table6)
results6

AGE_SEX_GROUP  \
AGE_SEX_GROUP                                      Between 19 and 30_F   
SEC_CONTRIBUTORY_CAUSE                                                   
ANIMAL                                                            0.18   
BICYCLE ADVANCING LEGALLY ON RED LIGHT                            0.06   
CELL PHONE USE OTHER THAN TEXTING                                 0.61   
DISREGARDING OTHER TRAFFIC SIGNS                                  0.55   
DISREGARDING ROAD MARKINGS                                        0.43   
DISREGARDING STOP SIGN                                            1.28   
DISREGARDING TRAFFIC SIGNALS                                      1.52   
DISREGARDING YIELD SIGN                                           0.00   
DISTRACTION - FROM INSIDE VEHICLE                                 1.40   
DISTRACTION - FROM OUTSIDE VEHICLE                                1.04   
DISTRACTION - OTHER ELECTRONIC DEVICE (NAVIGATI...                0.00   
DRIVING ON WRONG SIDE/WRONG WAY                                   0.43   
DRIVING SKILLS/KNOWLEDGE/EXPERIENCE                              13.29   
EQUIPMENT - VEHICLE CONDITION                                     0.91   
EVASIVE ACTION DUE TO ANIMAL, OBJECT, NONMOTORIST                 0.18   
EXCEEDING AUTHORIZED SPEED LIMIT                                  1.22   
EXCEEDING SAFE SPEED FOR CONDITIONS                               1.34   
FAILING TO REDUCE SPEED TO AVOID CRASH                           19.45   
FAILING TO YIELD RIGHT-OF-WAY                                    15.24   
FOLLOWING TOO CLOSELY                                            12.74   
HAD BEEN DRINKING (USE WHEN ARREST IS NOT MADE)                   0.37   
IMPROPER BACKING                                                  2.13   
IMPROPER LANE USAGE                                               5.61   
IMPROPER OVERTAKING/PASSING                                       5.06   
IMPROPER TURNING/NO SIGNAL                                        4.02   
MOTORCYCLE ADVANCING LEGALLY ON RED LIGHT                         0.00   
OBSTRUCTED CROSSWALKS                                             0.00   
OPERATING VEHICLE IN ERRATIC, RECKLESS, CARELES...                2.56   
PASSING STOPPED SCHOOL BUS                                        0.06   
PHYSICAL CONDITION OF DRIVER                                      0.49   
RELATED TO BUS STOP                                               0.18   
ROAD CONSTRUCTION/MAINTENANCE                                     0.24   
ROAD ENGINEERING/SURFACE/MARKING DEFECTS                          0.43   
TEXTING                                                           0.12   
TURNING RIGHT ON RED                                              0.06   
UNDER THE INFLUENCE OF ALCOHOL/DRUGS (USE WHEN ...                0.91   
VISION OBSCURED (SIGNS, TREE LIMBS, BUILDINGS, ...                1.28   
WEATHER                                                           4.57   
All                                                             100.00   

                                                                        \
AGE_SEX_GROUP                                      Between 19 and 30_M   
SEC_CONTRIBUTORY_CAUSE                                                   
ANIMAL                                                            0.10   
BICYCLE ADVANCING LEGALLY ON RED LIGHT                            0.19   
CELL PHONE USE OTHER THAN TEXTING                                 0.29   
DISREGARDING OTHER TRAFFIC SIGNS                                  0.48   
DISREGARDING ROAD MARKINGS                                        0.63   
DISREGARDING STOP SIGN                                            1.30   
DISREGARDING TRAFFIC SIGNALS                                      1.79   
DISREGARDING YIELD SIGN                                           0.10   
DISTRACTION - FROM INSIDE VEHICLE                                 1.26   
DISTRACTION - FROM OUTSIDE VEHICLE                          

,Chi-square test,results
0,Pearson Chi-square ( 259.0) =,450.7682
1,p-value =,0.0000
2,Cramer's V =,0.0685


In [109]:
# table6.to_excel(r'/Users/shellylin/repo_homlsl/sec_chi_square.xlsx', header = True)
# results6.to_excel(r'/Users/shellylin/repo_homlsl/sec_chi_square_result.xlsx', header = True)


In [113]:
table7, results7 = rp.crosstab(processed_data['PHYSICAL_CONDITION'], processed_data['AGE_SEX_GROUP'], 
                             prop='col', test='chi-square')

display(table7)
results7

# table7.to_excel(r'/Users/shellylin/repo_homlsl/physical_condition_chi_square.xlsx', header = True)
# results7.to_excel(r'/Users/shellylin/repo_homlsl/physical_condition_result.xlsx', header = True)

AGE_SEX_GROUP                      \
AGE_SEX_GROUP                Between 19 and 30_F Between 19 and 30_M   
PHYSICAL_CONDITION                                                     
EMOTIONAL                                   0.50                0.47   
FATIGUED/ASLEEP                             0.33                0.80   
HAD BEEN DRINKING                           0.08                0.20   
ILLNESS/FAINTED                             0.08                0.11   
IMPAIRED - ALCOHOL                          0.97                1.18   
IMPAIRED - ALCOHOL AND DRUGS                0.03                0.07   
IMPAIRED - DRUGS                            0.06                0.09   
MEDICATED                                   0.00                0.02   
NORMAL                                     97.01               96.36   
OTHER                                       0.17                0.33   
REMOVED BY EMS                              0.78                0.38   
All                                       100.00              100.00   

                                                                          \
AGE_SEX_GROUP                Middle Age 30 to 60_F Middle Age 30 to 60_M   
PHYSICAL_CONDITION                                                         
EMOTIONAL                                     0.52                  0.19   
FATIGUED/ASLEEP                               0.27                  0.54   
HAD BEEN DRINKING                             0.14                  0.06   
ILLNESS/FAINTED                               0.27                  0.22   
IMPAIRED - ALCOHOL                            0.54                  1.19   
IMPAIRED - ALCOHOL AND DRUGS                  0.00                  0.08   
IMPAIRED - DRUGS                              0.07                  0.15   
MEDICATED                                     0.05                  0.04   
NORMAL                                       97.34                 96.50   
OTHER                                         0.22                  0.39   
REMOVED BY EMS                                0.56                  0.64   
All                                         100.00                100.00   

                                                                         \
AGE_SEX_GROUP                Older than 60_F Older than 60_M Under 18_F   
PHYSICAL_CONDITION                                                        
EMOTIONAL                               0.56            0.07       0.44   
FATIGUED/ASLEEP                         0.31            0.38       0.44   
HAD BEEN DRINKING                       0.19            0.31       0.22   
ILLNESS/FAINTED                         0.31            0.38       0.22   
IMPAIRED - ALCOHOL                      0.43            0.93       0.44   
IMPAIRED - ALCOHOL AND DRUGS            0.00            0.00       0.00   
IMPAIRED - DRUGS                        0.12            0.17       0.00   
MEDICATED                               0.00            0.00       0.00   
NORMAL                                 96.90           96.09      97.14   
OTHER                                   0.50            0.97       0.44   
REMOVED BY EMS                          0.68            0.69       0.66   
All                                   100.00          100.00     100.00   

                                                 
AGE_SEX_GROUP                Under 18_M     All  
PHYSICAL_CONDITION                               
EMOTIONAL                          0.00    0.35  
FATIGUED/ASLEEP                    0.33    0.47  
HAD BEEN DRINKING                  0.16    0.14  
ILLNESS/FAINTED                    0.00    0.21  
IMPAIRED - ALCOHOL                 0.81    0.94  
IMPAIRED - ALCOHOL AND DRUGS       0.00    0.04  
IMPAIRED - DRUGS                   0.16    0.11  
MEDICATED                          0.00    0.03  
NORMAL                            97.40   96.72  
OTHER                              0.81    0.39  
REMOVED BY EMS                     0.

In [103]:
table8, results8 = rp.crosstab(processed_data['SAFETY_EQUIPMENT'], processed_data['AGE_SEX_GROUP'], 
                             prop='col', test='chi-square')

display(table8)
results8

table8.to_excel(r'/Users/shellylin/repo_homlsl/safety_equipment_chi_square.xlsx', header = True)
results8.to_excel(r'/Users/shellylin/repo_homlsl/safety_equipment_result.xlsx', header = True)


AGE_SEX_GROUP  \
AGE_SEX_GROUP                              Between 19 and 30_F   
SAFETY_EQUIPMENT                                                 
BICYCLE HELMET (PEDACYCLIST INVOLVED ONLY)                0.19   
CHILD RESTRAINT - TYPE UNKNOWN                            0.00   
DOT COMPLIANT MOTORCYCLE HELMET                           0.00   
HELMET NOT USED                                           0.52   
HELMET USED                                               0.15   
NONE PRESENT                                              4.87   
NOT DOT COMPLIANT MOTORCYCLE HELMET                       0.00   
SAFETY BELT NOT USED                                      0.64   
SAFETY BELT USED                                         93.59   
SHOULD/LAP BELT USED IMPROPERLY                           0.04   
WHEELCHAIR                                                0.00   
All                                                     100.00   

                                                                \
AGE_SEX_GROUP                              Between 19 and 30_M   
SAFETY_EQUIPMENT                                                 
BICYCLE HELMET (PEDACYCLIST INVOLVED ONLY)                0.41   
CHILD RESTRAINT - TYPE UNKNOWN                            0.00   
DOT COMPLIANT MOTORCYCLE HELMET                           0.63   
HELMET NOT USED                                           1.19   
HELMET USED                                               0.44   
NONE PRESENT                                              5.33   
NOT DOT COMPLIANT MOTORCYCLE HELMET                       0.03   
SAFETY BELT NOT USED                                      1.10   
SAFETY BELT USED                                         90.88   
SHOULD/LAP BELT USED IMPROPERLY                           0.00   
WHEELCHAIR                                                0.00   
All                                                     100.00   

                                                                  \
AGE_SEX_GROUP                              Middle Age 30 to 60_F   
SAFETY_EQUIPMENT                                                   
BICYCLE HELMET (PEDACYCLIST INVOLVED ONLY)                  0.21   
CHILD RESTRAINT - TYPE UNKNOWN                              0.00   
DOT COMPLIANT MOTORCYCLE HELMET                             0.00   
HELMET NOT USED                                             0.17   
HELMET USED                                                 0.21   
NONE PRESENT                                                4.79   
NOT DOT COMPLIANT MOTORCYCLE HELMET                         0.00   
SAFETY BELT NOT USED                                        0.69   
SAFETY BELT USED                                           93.86   
SHOULD/LAP BELT USED IMPROPERLY                             0.05   
WHEELCHAIR                                                  0.02   
All                                                       100.00   

                                                                  \
AGE_SEX_GROUP                              Middle Age 30 to 60_M   
SAFETY_EQUIPMENT                                                   
BICYCLE HELMET (PEDACYCLIST INVOLVED ONLY)                  0.33   
CHILD RESTRAINT - TYPE UNKNOWN                              0.00   
DOT COMPLIANT MOTORCYCLE HELMET                             0.20   
HELMET NOT USED                                             0.80   
HELMET USED                                                 0.23   
NONE PRESENT                                                5.16   
NOT DOT COMPLIANT MOTORCYCLE HELMET                         0.00   
SAFETY BELT NOT USED                                        0.83   
SAFETY BELT USED                                           92.42   
SHOULD/LAP BELT USED IMPROPERLY                             0.03   
WHEELCHAIR                                                  0.00   
All                                                       100.00   

                         

In [104]:
table9, results9 = rp.crosstab(processed_data['NUM_PASSENGERS'], processed_data['AGE_SEX_GROUP'], 
                             prop='col', test='chi-square')

display(table9)
results9

# table9.to_excel(r'/Users/shellylin/repo_homlsl/num_passengers_chi_square.xlsx', header = True)
# results9.to_excel(r'/Users/shellylin/repo_homlsl/num_passengers_result.xlsx', header = True)

AGE_SEX_GROUP                                            \
AGE_SEX_GROUP  Between 19 and 30_F Between 19 and 30_M Middle Age 30 to 60_F   
NUM_PASSENGERS                                                                 
0                            74.15               79.06                 77.48   
1                            16.73               15.00                 14.64   
10                            0.00                0.00                  0.03   
11                            0.00                0.00                  0.00   
12                            0.00                0.00                  0.02   
13                            0.00                0.00                  0.00   
19                            0.00                0.00                  0.02   
2                             5.63                3.94                  4.84   
29                            0.00                0.00                  0.00   
3                             2.33                1.52                  1.82   
32                            0.00                0.00                  0.02   
38                            0.00                0.00                  0.02   
4                             0.69                0.29                  0.64   
5                             0.22                0.12                  0.31   
6                             0.10                0.06                  0.10   
7                             0.10                0.00                  0.03   
8                             0.02                0.00                  0.03   
9                             0.02                0.00                  0.02   
All                         100.00              100.00                100.00   

                                                                      \
AGE_SEX_GROUP  Middle Age 30 to 60_M Older than 60_F Older than 60_M   
NUM_PASSENGERS                                                         
0                              83.36           85.01           85.09   
1                              12.02           11.16           11.70   
10                              0.00            0.00            0.00   
11                              0.00            0.00            0.02   
12                              0.00            0.00            0.00   
13                              0.01            0.00            0.00   
19                              0.00            0.00            0.00   
2                               2.80            2.40            1.92   
29                              0.00            0.00            0.02   
3                               1.16            1.04            0.96   
32                              0.00            0.00            0.00   
38                              0.00            0.00            0.00   
4                               0.46            0.28            0.20   
5                               0.11            0.03            0.11   
6                               0.03            0.00            0.00   
7                               0.04            0.03            0.00   
8                               0.00            0.03            0.00   
9                               0.01            0.00            0.00   
All                           100.00          100.00          100.00   

                                              
AGE_SEX_GROUP  Under 18_F Under 18_M     All  
NUM_PASSENGERS                                
0                   70.90      75.57   80.72  
1                   18.65      16.24   13.52  
10                   0.00       0.00    0.01  
11                   0.00       0.00    0.00  
12                   0.00       0.00    0.00  
13                   0.00       0.00    0.00  
19                   0.00       0.00    0.00  
2                    7.58       4.89    3.58  
29                   0.00       0.00    0.00  
3                    2.25       2.01    1.45  
32                   0.00       0.00    0.00  
38               

,Chi-square test,results
0,Pearson Chi-square ( 119.0) =,507.7428
1,p-value =,0.0000
2,Cramer's V =,0.0456


In [105]:
table10, results10 = rp.crosstab(processed_data['VEHICLE_TYPE'], processed_data['AGE_SEX_GROUP'], 
                             prop='col', test='chi-square')

display(table10)
results10

# table10.to_excel(r'/Users/shellylin/repo_homlsl/vehicle_type_chi_square.xlsx', header = True)
# results10.to_excel(r'/Users/shellylin/repo_homlsl/vehicle_type_result.xlsx', header = True)

AGE_SEX_GROUP                      \
AGE_SEX_GROUP                  Between 19 and 30_F Between 19 and 30_M   
VEHICLE_TYPE                                                             
ALL-TERRAIN VEHICLE (ATV)                     0.00                0.04   
AUTOCYCLE                                     0.00                0.02   
BUS OVER 15 PASS.                             0.74                0.82   
BUS UP TO 15 PASS.                            0.25                0.08   
FARM EQUIPMENT                                0.00                0.02   
MOPED OR MOTORIZED BICYCLE                    0.00                0.06   
MOTOR DRIVEN CYCLE                            0.00                0.06   
MOTORCYCLE (OVER 150CC)                       0.05                0.72   
OTHER                                         4.58                5.42   
OTHER VEHICLE WITH TRAILER                    0.00                0.16   
PASSENGER                                    77.99               71.98   
PICKUP                                        0.64                3.38   
SINGLE UNIT TRUCK WITH TRAILER                0.00                0.20   
SPORT UTILITY VEHICLE (SUV)                  12.32                9.39   
TRACTOR W/ SEMI-TRAILER                       0.05                1.03   
TRACTOR W/O SEMI-TRAILER                      0.00                0.16   
TRUCK - SINGLE UNIT                           0.37                2.30   
VAN/MINI-VAN                                  3.00                4.18   
All                                         100.00              100.00   

                                                                            \
AGE_SEX_GROUP                  Middle Age 30 to 60_F Middle Age 30 to 60_M   
VEHICLE_TYPE                                                                 
ALL-TERRAIN VEHICLE (ATV)                       0.00                  0.02   
AUTOCYCLE                                       0.03                  0.01   
BUS OVER 15 PASS.                               2.23                  1.94   
BUS UP TO 15 PASS.                              0.46                  0.16   
FARM EQUIPMENT                                  0.00                  0.02   
MOPED OR MOTORIZED BICYCLE                      0.00                  0.03   
MOTOR DRIVEN CYCLE                              0.05                  0.06   
MOTORCYCLE (OVER 150CC)                         0.03                  0.35   
OTHER                                           4.08                  5.62   
OTHER VEHICLE WITH TRAILER                      0.08                  0.23   
PASSENGER                                      68.15                 59.34   
PICKUP                                          0.74                  5.53   
SINGLE UNIT TRUCK WITH TRAILER                  0.00                  0.22   
SPORT UTILITY VEHICLE (SUV)                    18.07                 12.84   
TRACTOR W/ SEMI-TRAILER                         0.10                  2.56   
TRACTOR W/O SEMI-TRAILER                        0.00                  0.42   
TRUCK - SINGLE UNIT                             0.54                  3.88   
VAN/MINI-VAN                                    5.44                  6.76   
All                                           100.00                100.00   

                                                                           \
AGE_SEX_GROUP                  Older than 60_F Older than 60_M Under 18_F   
VEHICLE_TYPE                                                                
ALL-TERRAIN VEHICLE (ATV)                 0.00            0.00       0.00   
AUTOCYCLE                                 0.00            0.04       0.00   
BUS OVER 15 PASS.                         1.39            1.08       0.00   
BUS UP TO 15 PASS.                        0.24            0.30       0.00   
FARM EQUIPMENT                            0.00            0.00       0.00   
MOPED OR MOTORIZED BICYCLE                0.00            0.02       0.00  

,Chi-square test,results
0,Pearson Chi-square ( 119.0) =,2477.8057
1,p-value =,0.0000
2,Cramer's V =,0.1007


In [106]:
table11, results11 = rp.crosstab(processed_data['MANEUVER'], processed_data['AGE_SEX_GROUP'], 
                             prop='col', test='chi-square')

display(table11)
results11

# table11.to_excel(r'/Users/shellylin/repo_homlsl/maneuver_chi_square.xlsx', header = True)
# results11.to_excel(r'/Users/shellylin/repo_homlsl/maneuver_result.xlsx', header = True)

AGE_SEX_GROUP                      \
AGE_SEX_GROUP                      Between 19 and 30_F Between 19 and 30_M   
MANEUVER                                                                     
AVOIDING VEHICLES/OBJECTS                         0.50                0.76   
BACKING                                           2.78                3.49   
CHANGING LANES                                    2.13                2.81   
DIVERGING                                         0.00                0.00   
DRIVING WRONG WAY                                 0.22                0.18   
ENTER FROM DRIVE/ALLEY                            0.64                0.78   
ENTERING TRAFFIC LANE FROM PARKING                1.39                1.27   
LEAVING TRAFFIC LANE TO PARK                      0.45                0.29   
MERGING                                           0.99                0.94   
NEGOTIATING A CURVE                               0.20                0.21   
OTHER                                             5.43                5.19   
PARKED                                            0.00                0.00   
PASSING/OVERTAKING                                2.26                2.65   
SKIDDING/CONTROL LOSS                             0.72                0.94   
SLOW/STOP - LEFT TURN                             0.35                0.27   
SLOW/STOP - LOAD/UNLOAD                           0.07                0.10   
SLOW/STOP - RIGHT TURN                            0.20                0.18   
SLOW/STOP IN TRAFFIC                             10.09                8.35   
STARTING IN TRAFFIC                               0.69                0.59   
STRAIGHT AHEAD                                   57.08               57.52   
TURNING LEFT                                      9.14                8.43   
TURNING ON RED                                    0.12                0.04   
TURNING RIGHT                                     3.72                4.08   
U-TURN                                            0.84                0.94   
All                                             100.00              100.00   

                                                          \
AGE_SEX_GROUP                      Middle Age 30 to 60_F   
MANEUVER                                                   
AVOIDING VEHICLES/OBJECTS                           0.56   
BACKING                                             3.43   
CHANGING LANES                                      1.87   
DIVERGING                                           0.03   
DRIVING WRONG WAY                                   0.07   
ENTER FROM DRIVE/ALLEY                              0.51   
ENTERING TRAFFIC LANE FROM PARKING                  1.41   
LEAVING TRAFFIC LANE TO PARK                        0.51   
MERGING                                             0.79   
NEGOTIATING A CURVE                                 0.08   
OTHER                                               4.87   
PARKED                                              0.00   
PASSING/OVERTAKING                                  2.20   
SKIDDING/CONTROL LOSS                               0.36   
SLOW/STOP - LEFT TURN                               0.39   
SLOW/STOP - LOAD/UNLOAD                             0.20   
SLOW/STOP - RIGHT TURN                              0.30   
SLOW/STOP IN TRAFFIC                               14.67   
STARTING IN TRAFFIC                                 1.11   
STRAIGHT AHEAD                                     54.74   
TURNING LEFT                                        7.80   
TURNING ON RED                                      0.07   
TURNING RIGHT                                       3.43   
U-TURN                                              0.62   
All                                               100.00   

                                                                          \
AGE_SEX_GROUP                      Middle Age 30 to 60_M Older than 60_F   
MANEUVER               

,Chi-square test,results
0,Pearson Chi-square ( 161.0) =,1118.7271
1,p-value =,0.0000
2,Cramer's V =,0.0677


In [107]:
processed_data.isnull().sum()

POSTED_SPEED_LIMIT             0
TRAFFIC_CONTROL_DEVICE         0
DEVICE_CONDITION               0
WEATHER_CONDITION              0
LIGHTING_CONDITION             0
FIRST_CRASH_TYPE               0
TRAFFICWAY_TYPE                0
LANE_CNT                       0
ALIGNMENT                      0
ROADWAY_SURFACE_COND           0
ROAD_DEFECT                    0
INTERSECTION_RELATED_I         0
NOT_RIGHT_OF_WAY_I             0
HIT_AND_RUN_I                  0
PRIM_CONTRIBUTORY_CAUSE        0
SEC_CONTRIBUTORY_CAUSE     24805
STREET_DIRECTION               0
DOORING_I                      0
WORK_ZONE_I                    0
NUM_UNITS                      0
INJURIES_TOTAL                 0
CRASH_HOUR                     0
CRASH_DAY_OF_WEEK              0
CRASH_MONTH                    0
LATITUDE                       0
LONGITUDE                      0
NUM_PASSENGERS                 0
LIC_PLATE_STATE                0
VEHICLE_DEFECT                 0
VEHICLE_TYPE                   0
VEHICLE_US

In [108]:
result = pd.pivot_table(processed_data, index=['AGE'], columns='SEX', aggfunc='count')
print(result)

                    AGE_SEX_GROUP       AIRBAG_DEPLOYED       ALIGNMENT        \
SEX                             F     M               F     M         F     M   
AGE                                                                             
Between 19 and 30            4035  5125            4035  5125      4035  5125   
Middle Age 30 to 60          6100  9934            6100  9934      6100  9934   
Older than 60                2876  5639            2876  5639      2876  5639   
Under 18                      488   696             488   696       488   696   

                    BAC_RESULT VALUE       CELL_PHONE_USE        CITY        \
SEX                                F     M              F     M     F     M   
AGE                                                                           
Between 19 and 30               4035  5125           4035  5125  4035  5125   
Middle Age 30 to 60             6100  9934           6100  9934  6100  9934   
Older than 60                   2876 

In [54]:
trial1= pd.pivot_table(processed_data, index = ["SEX", "AGE"], aggfunc = 'count')
trial1

AGE_SEX_GROUP  AIRBAG_DEPLOYED  ALIGNMENT  \
SEX AGE                                                              
F   Between 19 and 30             5880             5880       5880   
    Middle Age 30 to 60           9217             9217       9217   
    Older than 60                 4900             4900       4900   
    Under 18                       684              684        684   
M   Between 19 and 30             7427             7427       7427   
    Middle Age 30 to 60          14595            14595      14595   
    Older than 60                 9278             9278       9278   
    Under 18                      1013             1013       1013   

                         BAC_RESULT VALUE  CELL_PHONE_USE   CITY  \
SEX AGE                                                            
F   Between 19 and 30                5880            5880   5880   
    Middle Age 30 to 60              9217            9217   9217   
    Older than 60                    4900            4900   4900   
    Under 18                          684             684    684   
M   Between 19 and 30                7427            7427   7427   
    Middle Age 30 to 60             14595           14595  14595   
    Older than 60                    9278            9278   9278   
    Under 18                         1013            1013   1013   

                         CRASH_DAY_OF_WEEK  CRASH_HOUR  CRASH_MONTH  \
SEX AGE                                                               
F   Between 19 and 30                 5880        5880         5880   
    Middle Age 30 to 60               9217        9217         9217   
    Older than 60                     4900        4900         4900   
    Under 18                           684         684          684   
M   Between 19 and 30                 7427        7427         7427   
    Middle Age 30 to 60              14595       14595        14595   
    Older than 60                     9278        9278         9278   
    Under 18                          1013        1013         1013   

                         DEVICE_CONDITION  DOORING_I  DRIVER_ACTION  \
SEX AGE                                                               
F   Between 19 and 30                5880       5880           5880   
    Middle Age 30 to 60              9217       9217           9217   
    Older than 60                    4900       4900           4900   
    Under 18                          684        684            684   
M   Between 19 and 30                7427       7427           7427   
    Middle Age 30 to 60             14595      14595          14595   
    Older than 60                    9278       9278           9278   
    Under 18                         1013       1013           1013   

                         DRIVER_VISION  EJECTION  EXCEED_SPEED_LIMIT_I  \
SEX AGE                                                                  
F   Between 19 and 30             5880      5880                  5880   
    Middle Age 30 to 60           9217      9217                  9217   
    Older than 60                 4900      4900                  4900   
    Under 18                       684       684                   684   
M   Between 19 and 30             7427      7427                  7427   
    Middle Age 30 to 60          14595     14595                 14595   
    Older than 60                 9278      9278                  9278   
    Under 18                      1013      1013                  1013   

                         FIRST_CONTACT_POINT  FIRST_CRASH_TYPE  HIT_AND_RUN_I  \
SEX AGE                                                                         
F   Between 19 and 30                   5880              5880           5880   
    Middle Age 30 to 60                 9217              9217           9217   
    Older than 60                       4900              4900           4900   
    Under 18                             684               684            6

In [55]:
trial2= pd.pivot_table(processed_data, index = ["SEX", "AGE"], aggfunc = 'count')
trial2

AGE_SEX_GROUP  AIRBAG_DEPLOYED  ALIGNMENT  \
SEX AGE                                                              
F   Between 19 and 30             5880             5880       5880   
    Middle Age 30 to 60           9217             9217       9217   
    Older than 60                 4900             4900       4900   
    Under 18                       684              684        684   
M   Between 19 and 30             7427             7427       7427   
    Middle Age 30 to 60          14595            14595      14595   
    Older than 60                 9278             9278       9278   
    Under 18                      1013             1013       1013   

                         BAC_RESULT VALUE  CELL_PHONE_USE   CITY  \
SEX AGE                                                            
F   Between 19 and 30                5880            5880   5880   
    Middle Age 30 to 60              9217            9217   9217   
    Older than 60                    4900            4900   4900   
    Under 18                          684             684    684   
M   Between 19 and 30                7427            7427   7427   
    Middle Age 30 to 60             14595           14595  14595   
    Older than 60                    9278            9278   9278   
    Under 18                         1013            1013   1013   

                         CRASH_DAY_OF_WEEK  CRASH_HOUR  CRASH_MONTH  \
SEX AGE                                                               
F   Between 19 and 30                 5880        5880         5880   
    Middle Age 30 to 60               9217        9217         9217   
    Older than 60                     4900        4900         4900   
    Under 18                           684         684          684   
M   Between 19 and 30                 7427        7427         7427   
    Middle Age 30 to 60              14595       14595        14595   
    Older than 60                     9278        9278         9278   
    Under 18                          1013        1013         1013   

                         DEVICE_CONDITION  DOORING_I  DRIVER_ACTION  \
SEX AGE                                                               
F   Between 19 and 30                5880       5880           5880   
    Middle Age 30 to 60              9217       9217           9217   
    Older than 60                    4900       4900           4900   
    Under 18                          684        684            684   
M   Between 19 and 30                7427       7427           7427   
    Middle Age 30 to 60             14595      14595          14595   
    Older than 60                    9278       9278           9278   
    Under 18                         1013       1013           1013   

                         DRIVER_VISION  EJECTION  EXCEED_SPEED_LIMIT_I  \
SEX AGE                                                                  
F   Between 19 and 30             5880      5880                  5880   
    Middle Age 30 to 60           9217      9217                  9217   
    Older than 60                 4900      4900                  4900   
    Under 18                       684       684                   684   
M   Between 19 and 30             7427      7427                  7427   
    Middle Age 30 to 60          14595     14595                 14595   
    Older than 60                 9278      9278                  9278   
    Under 18                      1013      1013                  1013   

                         FIRST_CONTACT_POINT  FIRST_CRASH_TYPE  HIT_AND_RUN_I  \
SEX AGE                                                                         
F   Between 19 and 30                   5880              5880           5880   
    Middle Age 30 to 60                 9217              9217           9217   
    Older than 60                       4900              4900           4900   
    Under 18                             684               684            6

In [56]:
# processed_data.to_excel(r'/Users/shellylin/repo_homlsl/export_dataframe.xlsx', index = False, header = True)

In [57]:
trial3= pd.pivot_table(processed_data, index = ["SEX", "AGE"], values = 'AIRBAG_DEPLOYED', aggfunc = 'count')
trial3

AIRBAG_DEPLOYED
SEX AGE                                 
F   Between 19 and 30               5880
    Middle Age 30 to 60             9217
    Older than 60                   4900
    Under 18                         684
M   Between 19 and 30               7427
    Middle Age 30 to 60            14595
    Older than 60                   9278
    Under 18                        1013

In [58]:
trial4= pd.crosstab(index = [processed_data["AIRBAG_DEPLOYED"],processed_data["ALIGNMENT"]], columns = [processed_data["SEX"], processed_data["AGE"]], margins=True)
trial4

SEX                                                       F  \
AGE                                       Between 19 and 30   
AIRBAG_DEPLOYED     ALIGNMENT                                 
DEPLOYED            CURVE ON GRADE                        2   
                    CURVE ON HILLCREST                    1   
                    CURVE, LEVEL                          6   
                    STRAIGHT AND LEVEL                  542   
                    STRAIGHT ON GRADE                    10   
                    STRAIGHT ON HILLCREST                 3   
NOT APPLICABLE      CURVE ON GRADE                        0   
                    CURVE ON HILLCREST                    0   
                    CURVE, LEVEL                          7   
                    STRAIGHT AND LEVEL                  871   
                    STRAIGHT ON GRADE                    14   
                    STRAIGHT ON HILLCREST                 2   
NOT DEPLOYED        CURVE ON GRADE                       12   
                    CURVE ON HILLCREST                    1   
                    CURVE, LEVEL                         36   
                    STRAIGHT AND LEVEL                 3840   
                    STRAIGHT ON GRADE                    59   
                    STRAIGHT ON HILLCREST                17   
UNABLE TO DETERMINE CURVE ON GRADE                        1   
                    CURVE ON HILLCREST                    0   
                    CURVE, LEVEL                          5   
                    STRAIGHT AND LEVEL                  442   
                    STRAIGHT ON GRADE                     5   
                    STRAIGHT ON HILLCREST                 4   
All                                                    5880   

SEX                                                                          \
AGE                                       Middle Age 30 to 60 Older than 60   
AIRBAG_DEPLOYED     ALIGNMENT                                                 
DEPLOYED            CURVE ON GRADE                          0             1   
                    CURVE ON HILLCREST                      2             0   
                    CURVE, LEVEL                            5             3   
                    STRAIGHT AND LEVEL                    553           117   
                    STRAIGHT ON GRADE                      13             1   
                    STRAIGHT ON HILLCREST                   0             2   
NOT APPLICABLE      CURVE ON GRADE                          3             0   
                    CURVE ON HILLCREST                      2             0   
                    CURVE, LEVEL                            7             1   
                    STRAIGHT AND LEVEL                   1485           542   
                    STRAIGHT ON GRADE                      24             5   
                    STRAIGHT ON HILLCREST                   3             2   
NOT DEPLOYED        CURVE ON GRADE                          4             2   
                    CURVE ON HILLCREST                      7             0   
                    CURVE, LEVEL                           44             8   
                    STRAIGHT AND LEVEL                   6336          2248   
                    STRAIGHT ON GRADE                      85            32   
                    STRAIGHT ON HILLCREST                  23             8   
UNABLE TO DETERMINE CURVE ON GRADE                          0             1   
                    CURVE ON HILLCREST                      1             0   
                    CURVE, LEVEL                            3            10   
                    STRAIGHT AND LEVEL                    609          1887   
                    STRAIGHT ON GRADE                       6            26   
                    STRAIGHT ON HILLCREST                   2             4   
All                                                      9217          4900   

SEX                   

In [59]:
trial5= pd.crosstab(index = [processed_data["AIRBAG_DEPLOYED"]], columns = [processed_data["SEX"], processed_data["AGE"]], margins=True)
trial5

SEX                                 F                                    \
AGE                 Between 19 and 30 Middle Age 30 to 60 Older than 60   
AIRBAG_DEPLOYED                                                           
DEPLOYED                          564                 573           124   
NOT APPLICABLE                    894                1524           550   
NOT DEPLOYED                     3965                6499          2298   
UNABLE TO DETERMINE               457                 621          1928   
All                              5880                9217          4900   

SEX                                          M                      \
AGE                 Under 18 Between 19 and 30 Middle Age 30 to 60   
AIRBAG_DEPLOYED                                                      
DEPLOYED                  62               743                 897   
NOT APPLICABLE           102              1199                2527   
NOT DEPLOYED             415              4862               10105   
UNABLE TO DETERMINE      105               623                1066   
All                      684              7427               14595   

SEX                                           All  
AGE                 Older than 60 Under 18         
AIRBAG_DEPLOYED                                    
DEPLOYED                      323       94   3380  
NOT APPLICABLE                977      152   7925  
NOT DEPLOYED                 3910      546  32600  
UNABLE TO DETERMINE          4068      221   9089  
All                          9278     1013  52994

In [60]:
trial6= pd.crosstab(index = [processed_data["ALIGNMENT"]], columns = [processed_data["SEX"], processed_data["AGE"]], margins=True)
trial6

SEX                                   F                                    \
AGE                   Between 19 and 30 Middle Age 30 to 60 Older than 60   
ALIGNMENT                                                                   
CURVE ON GRADE                       15                   7             4   
CURVE ON HILLCREST                    2                  12             0   
CURVE, LEVEL                         54                  59            22   
STRAIGHT AND LEVEL                 5695                8983          4794   
STRAIGHT ON GRADE                    88                 128            64   
STRAIGHT ON HILLCREST                26                  28            16   
All                                5880                9217          4900   

SEX                                            M                      \
AGE                   Under 18 Between 19 and 30 Middle Age 30 to 60   
ALIGNMENT                                                              
CURVE ON GRADE               1                29                  44   
CURVE ON HILLCREST           1                 8                   5   
CURVE, LEVEL                 6                65                 110   
STRAIGHT AND LEVEL         666              7191               14171   
STRAIGHT ON GRADE            5               101                 202   
STRAIGHT ON HILLCREST        5                33                  63   
All                        684              7427               14595   

SEX                                             All  
AGE                   Older than 60 Under 18         
ALIGNMENT                                            
CURVE ON GRADE                    8        1    109  
CURVE ON HILLCREST                4        0     32  
CURVE, LEVEL                     66       10    392  
STRAIGHT AND LEVEL             9071      989  51560  
STRAIGHT ON GRADE               104       12    704  
STRAIGHT ON HILLCREST            25        1    197  
All                            9278     1013  52994

In [61]:
combined = pd.concat([trial5, trial6], axis=0)
combined

SEX                                   F                                    \
AGE                   Between 19 and 30 Middle Age 30 to 60 Older than 60   
DEPLOYED                            564                 573           124   
NOT APPLICABLE                      894                1524           550   
NOT DEPLOYED                       3965                6499          2298   
UNABLE TO DETERMINE                 457                 621          1928   
All                                5880                9217          4900   
CURVE ON GRADE                       15                   7             4   
CURVE ON HILLCREST                    2                  12             0   
CURVE, LEVEL                         54                  59            22   
STRAIGHT AND LEVEL                 5695                8983          4794   
STRAIGHT ON GRADE                    88                 128            64   
STRAIGHT ON HILLCREST                26                  28            16   
All                                5880                9217          4900   

SEX                                            M                      \
AGE                   Under 18 Between 19 and 30 Middle Age 30 to 60   
DEPLOYED                    62               743                 897   
NOT APPLICABLE             102              1199                2527   
NOT DEPLOYED               415              4862               10105   
UNABLE TO DETERMINE        105               623                1066   
All                        684              7427               14595   
CURVE ON GRADE               1                29                  44   
CURVE ON HILLCREST           1                 8                   5   
CURVE, LEVEL                 6                65                 110   
STRAIGHT AND LEVEL         666              7191               14171   
STRAIGHT ON GRADE            5               101                 202   
STRAIGHT ON HILLCREST        5                33                  63   
All                        684              7427               14595   

SEX                                             All  
AGE                   Older than 60 Under 18         
DEPLOYED                        323       94   3380  
NOT APPLICABLE                  977      152   7925  
NOT DEPLOYED                   3910      546  32600  
UNABLE TO DETERMINE            4068      221   9089  
All                            9278     1013  52994  
CURVE ON GRADE                    8        1    109  
CURVE ON HILLCREST                4        0     32  
CURVE, LEVEL                     66       10    392  
STRAIGHT AND LEVEL             9071      989  51560  
STRAIGHT ON GRADE               104       12    704  
STRAIGHT ON HILLCREST            25        1    197  
All                            9278     1013  52994

In [62]:
trial7 = pd.crosstab([processed_data.SEX, processed_data.AGE],
                    [processed_data.AIRBAG_DEPLOYED, processed_data.ALIGNMENT], margins = False)
trial7

AIRBAG_DEPLOYED               DEPLOYED                                  \
ALIGNMENT               CURVE ON GRADE CURVE ON HILLCREST CURVE, LEVEL   
SEX AGE                                                                  
F   Between 19 and 30                2                  1            6   
    Middle Age 30 to 60              0                  2            5   
    Older than 60                    1                  0            3   
    Under 18                         0                  0            3   
M   Between 19 and 30                4                  3           15   
    Middle Age 30 to 60              7                  0            9   
    Older than 60                    2                  1            9   
    Under 18                         0                  0            2   

AIRBAG_DEPLOYED                                               \
ALIGNMENT               STRAIGHT AND LEVEL STRAIGHT ON GRADE   
SEX AGE                                                        
F   Between 19 and 30                  542                10   
    Middle Age 30 to 60                553                13   
    Older than 60                      117                 1   
    Under 18                            58                 1   
M   Between 19 and 30                  701                14   
    Middle Age 30 to 60                863                 8   
    Older than 60                      304                 4   
    Under 18                            90                 2   

AIRBAG_DEPLOYED                               NOT APPLICABLE  \
ALIGNMENT               STRAIGHT ON HILLCREST CURVE ON GRADE   
SEX AGE                                                        
F   Between 19 and 30                       3              0   
    Middle Age 30 to 60                     0              3   
    Older than 60                           2              0   
    Under 18                                0              0   
M   Between 19 and 30                       6              5   
    Middle Age 30 to 60                    10              9   
    Older than 60                           3              0   
    Under 18                                0              0   

AIRBAG_DEPLOYED                                                             \
ALIGNMENT               CURVE ON HILLCREST CURVE, LEVEL STRAIGHT AND LEVEL   
SEX AGE                                                                      
F   Between 19 and 30                    0            7                871   
    Middle Age 30 to 60                  2            7               1485   
    Older than 60                        0            1                542   
    Under 18                             0            0                101   
M   Between 19 and 30                    2           13               1163   
    Middle Age 30 to 60                  1           15               2459   
    Older than 60                        0            7                958   
    Under 18                             0            2                150   

AIRBAG_DEPLOYED                                                  \
ALIGNMENT               STRAIGHT ON GRADE STRAIGHT ON HILLCREST   
SEX AGE                                                           
F   Between 19 and 30                  14                     2   
    Middle Age 30 to 60                24                     3   
    Older than 60                       5                     2   
    Under 18                            1                     0   
M   Between 19 and 30                  10                     6   
    Middle Age 30 to 60                39                     4   
    Older than 60                      10                     2   
    Under 18                            0                     0   

AIRBAG_DEPLOYED           NOT DEPLOYED                                  \
ALIGNMENT               CURVE ON GRADE CURVE ON HILLCREST CURVE, LEVEL   
SEX AGE                       

In [63]:
trial8= pd.crosstab(index = [processed_data["INJURY"], processed_data["AIRBAG_DEPLOYED"]], columns = [processed_data["SEX"], processed_data["AGE"]], margins=True)
trial8

SEX                                             F                      \
AGE                             Between 19 and 30 Middle Age 30 to 60   
INJURY      AIRBAG_DEPLOYED                                             
INJURED     DEPLOYED                          246                 258   
            NOT APPLICABLE                     60                 116   
            NOT DEPLOYED                      335                 548   
            UNABLE TO DETERMINE               188                 202   
NOT INJURED DEPLOYED                          318                 315   
            NOT APPLICABLE                    834                1408   
            NOT DEPLOYED                     3630                5951   
            UNABLE TO DETERMINE               269                 419   
All                                          5880                9217   

SEX                                                                    M  \
AGE                             Older than 60 Under 18 Between 19 and 30   
INJURY      AIRBAG_DEPLOYED                                                
INJURED     DEPLOYED                       63       26               295   
            NOT APPLICABLE                 50        3               127   
            NOT DEPLOYED                  144       37               359   
            UNABLE TO DETERMINE           210       71               192   
NOT INJURED DEPLOYED                       61       36               448   
            NOT APPLICABLE                500       99              1072   
            NOT DEPLOYED                 2154      378              4503   
            UNABLE TO DETERMINE          1718       34               431   
All                                      4900      684              7427   

SEX                                                                         \
AGE                             Middle Age 30 to 60 Older than 60 Under 18   
INJURY      AIRBAG_DEPLOYED                                                  
INJURED     DEPLOYED                            379           127       43   
            NOT APPLICABLE                      240            75       13   
            NOT DEPLOYED                        789           252       49   
            UNABLE TO DETERMINE                 358           425      142   
NOT INJURED DEPLOYED                            518           196       51   
            NOT APPLICABLE                     2287           902      139   
            NOT DEPLOYED                       9316          3658      497   
            UNABLE TO DETERMINE                 708          3643       79   
All                                           14595          9278     1013   

SEX                                All  
AGE                                     
INJURY      AIRBAG_DEPLOYED             
INJURED     DEPLOYED              1437  
            NOT APPLICABLE         684  
            NOT DEPLOYED          2513  
            UNABLE TO DETERMINE   1788  
NOT INJURED DEPLOYED              1943  
            NOT APPLICABLE        7241  
            NOT DEPLOYED         30087  
            UNABLE TO DETERMINE   7301  
All                              52994

In [64]:
trial9= pd.crosstab(index = [processed_data["INJURY"], processed_data["FIRST_CRASH_TYPE"]], columns = [processed_data["SEX"], processed_data["AGE"]], margins=True)
trial9

SEX                                                      F  \
AGE                                      Between 19 and 30   
INJURY      FIRST_CRASH_TYPE                                 
INJURED     ANGLE                                      178   
            FIXED OBJECT                                29   
            HEAD ON                                     27   
            OTHER NONCOLLISION                           0   
            OTHER OBJECT                                 2   
            OVERTURNED                                   1   
            PARKED MOTOR VEHICLE                        28   
            PEDALCYCLIST                                54   
            PEDESTRIAN                                 140   
            REAR END                                   145   
            REAR TO FRONT                                3   
            REAR TO SIDE                                 0   
            SIDESWIPE OPPOSITE DIRECTION                 6   
            SIDESWIPE SAME DIRECTION                    51   
            TRAIN                                        1   
            TURNING                                    164   
NOT INJURED ANGLE                                      716   
            ANIMAL                                       2   
            FIXED OBJECT                               103   
            HEAD ON                                     47   
            OTHER NONCOLLISION                           9   
            OTHER OBJECT                                21   
            OVERTURNED                                   1   
            PARKED MOTOR VEHICLE                       349   
            PEDALCYCLIST                                29   
            PEDESTRIAN                                  43   
            REAR END                                  1581   
            REAR TO FRONT                               36   
            REAR TO REAR                                 6   
            REAR TO SIDE                                30   
            SIDESWIPE OPPOSITE DIRECTION                79   
            SIDESWIPE SAME DIRECTION                   929   
            TRAIN                                        0   
            TURNING                                   1070   
All                                                   5880   

SEX                                                                         \
AGE                                      Middle Age 30 to 60 Older than 60   
INJURY      FIRST_CRASH_TYPE                                                 
INJURED     ANGLE                                        235            69   
            FIXED OBJECT                                  25            12   
            HEAD ON                                       22             3   
            OTHER NONCOLLISION                             4             1   
            OTHER OBJECT                                   2             5   
            OVERTURNED                                     0             0   
            PARKED MOTOR VEHICLE                          26            15   
            PEDALCYCLIST                                  73            49   
            PEDESTRIAN                                   190           139   
            REAR END                                     244            80   
            REAR TO FRONT                                  3             2   
            REAR TO SIDE                                   0             2   
            SIDESWIPE OPPOSITE DIRECTION                  25             5   
            SIDESWIPE SAME DIRECTION                      75            20   
            TRAIN                                          0             0   
            TURNING                                      200            65   
NOT INJURED ANGLE                                       1119           562   
            ANIMAL                                         8             5   
     

In [65]:
trial10= pd.crosstab(index = [processed_data["INJURY"], processed_data["TRAFFIC_CONTROL_DEVICE"]], columns = [processed_data["SEX"], processed_data["AGE"]], margins=True)
trial10

SEX                                                  F                      \
AGE                                  Between 19 and 30 Middle Age 30 to 60   
INJURY      TRAFFIC_CONTROL_DEVICE                                           
INJURED     FLASHING CONTROL SIGNAL                  0                   1   
            LANE USE MARKING                         2                   2   
            NO CONTROLS                            322                 436   
            OTHER                                    4                   8   
            OTHER RAILROAD CROSSING                  0                   0   
            OTHER REG. SIGN                          3                   2   
            OTHER WARNING SIGN                       0                   0   
            PEDESTRIAN CROSSING SIGN                 0                   1   
            POLICE/FLAGMAN                           0                   0   
            RAILROAD CROSSING GATE                   2                   1   
            SCHOOL ZONE                              2                   1   
            STOP SIGN/FLASHER                      137                 173   
            TRAFFIC SIGNAL                         351                 491   
            YIELD                                    6                   8   
NOT INJURED BICYCLE CROSSING SIGN                    0                   0   
            DELINEATORS                              0                   2   
            FLASHING CONTROL SIGNAL                  0                   2   
            LANE USE MARKING                        14                  19   
            NO CONTROLS                           2620                4117   
            NO PASSING                               0                   0   
            OTHER                                   21                  41   
            OTHER RAILROAD CROSSING                  3                   1   
            OTHER REG. SIGN                          6                   4   
            OTHER WARNING SIGN                      10                   5   
            PEDESTRIAN CROSSING SIGN                 2                   2   
            POLICE/FLAGMAN                           3                   9   
            RAILROAD CROSSING GATE                   5                   5   
            RR CROSSING SIGN                         0                   2   
            SCHOOL ZONE                              0                   4   
            STOP SIGN/FLASHER                      696                1033   
            TRAFFIC SIGNAL                        1661                2830   
            YIELD                                   10                  17   
All                                               5880                9217   

SEX                                                                         M  \
AGE                                  Older than 60 Under 18 Between 19 and 30   
INJURY      TRAFFIC_CONTROL_DEVICE                                              
INJURED     FLASHING CONTROL SIGNAL              0        0                 1   
            LANE USE MARKING                     2        0                 3   
            NO CONTROLS                        182       49               427   
            OTHER                                4        0                 4   
            OTHER RAILROAD CROSSING              0        0                 1   
            OTHER REG. SIGN                      1        0                 1   
            OTHER WARNING SIGN                   0        0                 1   
            PEDESTRIAN CROSSING SIGN             3        1                 0   
            POLICE/FLAGMAN                       1        0                 0   
            RAILROAD CROSSING GATE               0        0                 0   
            SCHOOL ZONE                          1        0                 1   
            STOP SIGN/FLASHER                   73       

In [66]:
trial11= pd.crosstab(index = [processed_data["INJURY"], processed_data["TRAFFICWAY_TYPE"]], columns = [processed_data["SEX"], processed_data["AGE"]], margins=True)
trial11

SEX                                         F                      \
AGE                         Between 19 and 30 Middle Age 30 to 60   
INJURY      TRAFFICWAY_TYPE                                         
INJURED     DIVIDED                       226                 260   
            NOT_DIVIDED                   603                 864   
NOT INJURED DIVIDED                      1141                1870   
            NOT_DIVIDED                  3910                6223   
All                                      5880                9217   

SEX                                                                M  \
AGE                         Older than 60 Under 18 Between 19 and 30   
INJURY      TRAFFICWAY_TYPE                                            
INJURED     DIVIDED                    99       30               270   
            NOT_DIVIDED               368      107               703   
NOT INJURED DIVIDED                  1104      144              1442   
            NOT_DIVIDED              3329      403              5012   
All                                  4900      684              7427   

SEX                                                                       All  
AGE                         Middle Age 30 to 60 Older than 60 Under 18         
INJURY      TRAFFICWAY_TYPE                                                    
INJURED     DIVIDED                         472           237       68   1662  
            NOT_DIVIDED                    1294           642      179   4760  
NOT INJURED DIVIDED                        2821          1874      166  10562  
            NOT_DIVIDED                   10008          6525      600  36010  
All                                       14595          9278     1013  52994

In [67]:
trial12= pd.crosstab(index = [processed_data["INJURY"], processed_data["ROADWAY_SURFACE_COND"]], columns = [processed_data["SEX"], processed_data["AGE"]], margins=True)
trial12

SEX                                              F                      \
AGE                              Between 19 and 30 Middle Age 30 to 60   
INJURY      ROADWAY_SURFACE_COND                                         
INJURED     DRY                                645                 866   
            ICE                                  8                   9   
            NO DEFECTS                          18                  22   
            OTHER                                3                   6   
            SAND, MUD, DIRT                      0                   2   
            SNOW OR SLUSH                       21                  32   
            WET                                134                 187   
NOT INJURED DRY                               3779                6152   
            ICE                                 50                  80   
            NO DEFECTS                         224                 403   
            OTHER                               10                  23   
            SAND, MUD, DIRT                      0                   2   
            SNOW OR SLUSH                      181                 280   
            WET                                807                1153   
All                                           5880                9217   

SEX                                                                     M  \
AGE                              Older than 60 Under 18 Between 19 and 30   
INJURY      ROADWAY_SURFACE_COND                                            
INJURED     DRY                            358      109               757   
            ICE                              1        2                13   
            NO DEFECTS                      14        2                24   
            OTHER                            1        0                 2   
            SAND, MUD, DIRT                  0        0                 0   
            SNOW OR SLUSH                    9        1                25   
            WET                             84       23               152   
NOT INJURED DRY                           3463      417              4711   
            ICE                             25        7                70   
            NO DEFECTS                     255       21               325   
            OTHER                            8        0                12   
            SAND, MUD, DIRT                  1        0                 1   
            SNOW OR SLUSH                  129       18               282   
            WET                            552       84              1053   
All                                       4900      684              7427   

SEX                                                                          \
AGE                              Middle Age 30 to 60 Older than 60 Under 18   
INJURY      ROADWAY_SURFACE_COND                                              
INJURED     DRY                                 1332           677      185   
            ICE                                   18             8        1   
            NO DEFECTS                            49            42        5   
            OTHER                                  7             3        0   
            SAND, MUD, DIRT                        0             0        0   
            SNOW OR SLUSH                         56            21        1   
            WET                                  304           128       55   
NOT INJURED DRY                                 9486          6371      575   
            ICE                                  142            55        5   
            NO DEFECTS                           629           447       32   
            OTHER                                 29            15        2   
            SAND, MUD, DIRT                        7             4        0   
            SNOW OR SLUSH                        517           309       28   
            WET 

In [68]:
trial13= pd.crosstab(index = [processed_data["INJURY"], processed_data["ROAD_DEFECT"]], columns = [processed_data["SEX"], processed_data["AGE"]], margins=True)
trial13

SEX                                           F                      \
AGE                           Between 19 and 30 Middle Age 30 to 60   
INJURY      ROAD_DEFECT                                               
INJURED     CLEAR                            96                 121   
            DEBRIS ON ROADWAY                 2                   2   
            NO DEFECTS                      724                 987   
            OTHER                             3                   7   
            RUT, HOLES                        1                   1   
            SHOULDER DEFECT                   3                   5   
            WORN SURFACE                      0                   1   
NOT INJURED CLEAR                           685                1126   
            DEBRIS ON ROADWAY                 4                   6   
            NO DEFECTS                     4274                6802   
            OTHER                            17                  45   
            RUT, HOLES                       39                  63   
            SHOULDER DEFECT                  16                  19   
            WORN SURFACE                     16                  32   
All                                        5880                9217   

SEX                                                                  M  \
AGE                           Older than 60 Under 18 Between 19 and 30   
INJURY      ROAD_DEFECT                                                  
INJURED     CLEAR                        58       12               113   
            DEBRIS ON ROADWAY             2        0                 0   
            NO DEFECTS                  403      125               849   
            OTHER                         3        0                 4   
            RUT, HOLES                    0        0                 3   
            SHOULDER DEFECT               0        0                 2   
            WORN SURFACE                  1        0                 2   
NOT INJURED CLEAR                       644       74               908   
            DEBRIS ON ROADWAY             1        2                 5   
            NO DEFECTS                 3737      465              5408   
            OTHER                        15        0                36   
            RUT, HOLES                   15        4                50   
            SHOULDER DEFECT              12        1                20   
            WORN SURFACE                  9        1                27   
All                                    4900      684              7427   

SEX                                                                       \
AGE                           Middle Age 30 to 60 Older than 60 Under 18   
INJURY      ROAD_DEFECT                                                    
INJURED     CLEAR                             182           120       36   
            DEBRIS ON ROADWAY                   2             0        0   
            NO DEFECTS                       1546           745      208   
            OTHER                              19             8        1   
            RUT, HOLES                          4             3        0   
            SHOULDER DEFECT                     4             0        0   
            WORN SURFACE                        9             3        2   
NOT INJURED CLEAR                            1713          1177      101   
            DEBRIS ON ROADWAY                   9             5        0   
            NO DEFECTS                      10793          7094      648   
            OTHER                              87            35        6   
            RUT, HOLES                        137            44        4   
            SHOULDER DEFECT                    32            17        2   
            WORN SURFACE                       58            27        5   
All                                         14595          9278     1013   

SEX                

In [69]:
trial14= pd.crosstab(index = [processed_data["INJURY"], processed_data["NOT_RIGHT_OF_WAY_I"]], columns = [processed_data["SEX"], processed_data["AGE"]], margins=True)
trial14

SEX                                            F                      \
AGE                            Between 19 and 30 Middle Age 30 to 60   
INJURY      NOT_RIGHT_OF_WAY_I                                         
INJURED     N                                812                1108   
            Y                                 17                  16   
NOT INJURED N                               4904                7837   
            Y                                147                 256   
All                                         5880                9217   

SEX                                                                   M  \
AGE                            Older than 60 Under 18 Between 19 and 30   
INJURY      NOT_RIGHT_OF_WAY_I                                            
INJURED     N                            456      135               950   
            Y                             11        2                23   
NOT INJURED N                           4254      524              6283   
            Y                            179       23               171   
All                                     4900      684              7427   

SEX                                                                        \
AGE                            Middle Age 30 to 60 Older than 60 Under 18   
INJURY      NOT_RIGHT_OF_WAY_I                                              
INJURED     N                                 1720           861      246   
            Y                                   46            18        1   
NOT INJURED N                                12420          8075      743   
            Y                                  409           324       23   
All                                          14595          9278     1013   

SEX                               All  
AGE                                    
INJURY      NOT_RIGHT_OF_WAY_I         
INJURED     N                    6288  
            Y                     134  
NOT INJURED N                   45040  
            Y                    1532  
All                             52994

In [70]:
trial15= pd.crosstab(index = [processed_data["INJURY"], processed_data["HIT_AND_RUN_I"]], columns = [processed_data["SEX"], processed_data["AGE"]], margins=True)
trial15

SEX                                       F                                    \
AGE                       Between 19 and 30 Middle Age 30 to 60 Older than 60   
INJURY      HIT_AND_RUN_I                                                       
INJURED     N                           707                 995           346   
            Y                           122                 129           121   
NOT INJURED N                          4381                6970          3018   
            Y                           670                1123          1415   
All                                    5880                9217          4900   

SEX                                                M                      \
AGE                       Under 18 Between 19 and 30 Middle Age 30 to 60   
INJURY      HIT_AND_RUN_I                                                  
INJURED     N                  111               837                1543   
            Y                   26               136                 223   
NOT INJURED N                  471              5686               11412   
            Y                   76               768                1417   
All                            684              7427               14595   

SEX                                                 All  
AGE                       Older than 60 Under 18         
INJURY      HIT_AND_RUN_I                                
INJURED     N                       539      195   5273  
            Y                       340       52   1149  
NOT INJURED N                      5310      690  37938  
            Y                      3089       76   8634  
All                                9278     1013  52994

In [71]:
#  pd.set_option('display.max_rows', None)  # prevent column output trancation
trial16= pd.crosstab(index = [processed_data["INJURY"], processed_data["PRIM_CONTRIBUTORY_CAUSE"]], columns = [processed_data["AGE"], processed_data["SEX"]], margins=True)
trial16

AGE                                                            Between 19 and 30  \
SEX                                                                            F   
INJURY      PRIM_CONTRIBUTORY_CAUSE                                                
INJURED     ANIMAL                                                             1   
            BICYCLE ADVANCING LEGALLY ON RED LIGHT                             0   
            CELL PHONE USE OTHER THAN TEXTING                                  0   
            DISREGARDING OTHER TRAFFIC SIGNS                                   7   
            DISREGARDING ROAD MARKINGS                                         1   
...                                                                          ...   
NOT INJURED UNABLE TO DETERMINE                                             1421   
            UNDER THE INFLUENCE OF ALCOHOL/DRUGS (USE WHEN ...                24   
            VISION OBSCURED (SIGNS, TREE LIMBS, BUILDINGS, ...                34   
            WEATHER                                                           99   
All                                                                         5880   

AGE                                                                   \
SEX                                                                M   
INJURY      PRIM_CONTRIBUTORY_CAUSE                                    
INJURED     ANIMAL                                                 0   
            BICYCLE ADVANCING LEGALLY ON RED LIGHT                 0   
            CELL PHONE USE OTHER THAN TEXTING                      2   
            DISREGARDING OTHER TRAFFIC SIGNS                       8   
            DISREGARDING ROAD MARKINGS                             0   
...                                                              ...   
NOT INJURED UNABLE TO DETERMINE                                 1728   
            UNDER THE INFLUENCE OF ALCOHOL/DRUGS (USE WHEN ...    37   
            VISION OBSCURED (SIGNS, TREE LIMBS, BUILDINGS, ...    40   
            WEATHER                                              172   
All                                                             7427   

AGE                                                            Middle Age 30 to 60  \
SEX                                                                              F   
INJURY      PRIM_CONTRIBUTORY_CAUSE                                                  
INJURED     ANIMAL                                                               1   
            BICYCLE ADVANCING LEGALLY ON RED LIGHT                               0   
            CELL PHONE USE OTHER THAN TEXTING                                    2   
            DISREGARDING OTHER TRAFFIC SIGNS                                     5   
            DISREGARDING ROAD MARKINGS                                           3   
...                                                                            ...   
NOT INJURED UNABLE TO DETERMINE                                               2465   
            UNDER THE INFLUENCE OF ALCOHOL/DRUGS (USE WHEN ...                  24   
            VISION OBSCURED (SIGNS, TREE LIMBS, BUILDINGS, ...                  56   
            WEATHER                                                            139   
All                                                                           9217   

AGE                                                                    \
SEX                                                                 M   
INJURY      PRIM_CONTRIBUTORY_CAUSE                                     
INJURED     ANIMAL                                                  4   
            BICYCLE ADVANCING LEGALLY ON RED LIGHT                  1   
            CELL PHONE USE OTHER THAN TEXTING                       3   
            DISREGARDING OTHER TRAFFIC SIGNS                       19   
            DISREGARDING ROAD MARKINGS                              6   
...                      

In [37]:
trial16= pd.crosstab(index = [processed_data["INJURY"], processed_data["PERSON_TYPE"]], columns = [processed_data["AGE"], processed_data["SEX"]], margins=True)
trial16

AGE                             Between 19 and 30       Middle Age         \
SEX                                             F     M          F      M   
INJURY      PERSON_TYPE                                                     
INJURED     BICYCLE                            34    55         30     98   
            DRIVER                            679   826        967   1501   
            NON-CONTACT VEHICLE                 0     1          0      0   
            NON-MOTOR VEHICLE                   0     0          0      2   
            PEDESTRIAN                        116    91        127    165   
NOT INJURED BICYCLE                            15    47         17     52   
            DRIVER                           5007  6372       8016  12720   
            NON-CONTACT VEHICLE                 0     0          0      0   
            NON-MOTOR VEHICLE                   0     3          2     13   
            PEDESTRIAN                         29    32         58     44   
All                                          5880  7427       9217  14595   

AGE                             Older       Under 18          All  
SEX                                 F     M        F     M         
INJURY      PERSON_TYPE                                            
INJURED     BICYCLE                 6    30        6    55    314  
            DRIVER                370   769       67   114   5293  
            NON-CONTACT VEHICLE     0     0        0     0      1  
            NON-MOTOR VEHICLE       0     1        0     0      3  
            PEDESTRIAN             91    79       64    78    811  
NOT INJURED BICYCLE                 3    42        3    16    195  
            DRIVER               4392  8313      531   728  46079  
            NON-CONTACT VEHICLE     2     3        0     0      5  
            NON-MOTOR VEHICLE       3     5        1     1     28  
            PEDESTRIAN             33    36       12    21    265  
All                              4900  9278      684  1013  52994

In [38]:
trial17= pd.crosstab(index = [processed_data["INJURY"], processed_data["VEHICLE_TYPE"]], columns = [processed_data["AGE"], processed_data["SEX"]], margins=True)
trial17

AGE                                              Between 19 and 30        \
SEX                                                              F     M   
INJURY      VEHICLE_TYPE                                                   
INJURED     ALL-TERRAIN VEHICLE (ATV)                            0     3   
            AUTOCYCLE                                            0     0   
            BUS OVER 15 PASS.                                    3     4   
            BUS UP TO 15 PASS.                                   0     0   
            FARM EQUIPMENT                                       0     1   
            MOPED OR MOTORIZED BICYCLE                           0     2   
            MOTOR DRIVEN CYCLE                                   0     1   
            MOTORCYCLE (OVER 150CC)                              2    33   
            OTHER                                              163   165   
            OTHER VEHICLE WITH TRAILER                           0     1   
            PASSENGER                                          565   617   
            PICKUP                                               4    27   
            SINGLE UNIT TRUCK WITH TRAILER                       0     0   
            SPORT UTILITY VEHICLE (SUV)                         71    78   
            TRACTOR W/ SEMI-TRAILER                              1     5   
            TRACTOR W/O SEMI-TRAILER                             0     0   
            TRUCK - SINGLE UNIT                                  0    11   
            VAN/MINI-VAN                                        20    25   
NOT INJURED ALL-TERRAIN VEHICLE (ATV)                            1     0   
            AUTOCYCLE                                            0     1   
            BUS OVER 15 PASS.                                   42    45   
            BUS UP TO 15 PASS.                                  13     5   
            FARM EQUIPMENT                                       0     0   
            MOPED OR MOTORIZED BICYCLE                           0     2   
            MOTOR DRIVEN CYCLE                                   1     3   
            MOTORCYCLE (OVER 150CC)                              0    13   
            OTHER                                              181   329   
            OTHER VEHICLE WITH TRAILER                           4    13   
            PASSENGER                                         3958  4671   
            PICKUP                                              35   208   
            SINGLE UNIT TRUCK WITH TRAILER                       0    12   
            SPORT UTILITY VEHICLE (SUV)                        637   604   
            TRACTOR W/ SEMI-TRAILER                              3    82   
            TRACTOR W/O SEMI-TRAILER                             0    12   
            TRUCK - SINGLE UNIT                                 26   177   
            VAN/MINI-VAN                                       150   277   
            3-WHEELED MOTORCYCLE (2 REAR WHEELS)                 0     0   
All                                                           5880  7427   

AGE                                              Middle Age        Older  \
SEX                                                       F      M     F   
INJURY      VEHICLE_TYPE                                                   
INJURED     ALL-TERRAIN VEHICLE (ATV)                     1      2     0   
            AUTOCYCLE                                     0      2     0   
            BUS OVER 15 PASS.                            21     25     6   
            BUS UP TO 15 PASS.                            4      3     3   
            FARM EQUIPMENT                                0      1     0   
            MOPED OR MOTORIZED BICYCLE                    1      4     0   
            MOTOR DRIVEN CYCLE                            3      3     1   
            MOTORCYCLE (OVER 150CC)                       1     36     0   
            OTHER                              

In [39]:
import researchpy as rp
table3, results3 = rp.crosstab(processed_data['AGE_SEX_GROUP'], processed_data['PRIM_CONTRIBUTORY_CAUSE'], 
                             prop='col', test='chi-square')

display(table3)
results3


PRIM_CONTRIBUTORY_CAUSE  \
PRIM_CONTRIBUTORY_CAUSE                  ANIMAL   
AGE_SEX_GROUP                                     
Between 19 and 30F                        13.95   
Between 19 and 30M                         9.30   
Middle AgeF                               30.23   
Middle AgeM                               25.58   
OlderF                                     9.30   
OlderM                                     2.33   
Under 18F                                  0.00   
Under 18M                                  9.30   
All                                      100.00   

                                                                \
PRIM_CONTRIBUTORY_CAUSE BICYCLE ADVANCING LEGALLY ON RED LIGHT   
AGE_SEX_GROUP                                                    
Between 19 and 30F                                        12.5   
Between 19 and 30M                                        12.5   
Middle AgeF                                               37.5   
Middle AgeM                                               12.5   
OlderF                                                    12.5   
OlderM                                                    12.5   
Under 18F                                                  0.0   
Under 18M                                                  0.0   
All                                                      100.0   

                                                           \
PRIM_CONTRIBUTORY_CAUSE CELL PHONE USE OTHER THAN TEXTING   
AGE_SEX_GROUP                                               
Between 19 and 30F                                  20.24   
Between 19 and 30M                                  17.86   
Middle AgeF                                         11.90   
Middle AgeM                                         30.95   
OlderF                                               7.14   
OlderM                                              10.71   
Under 18F                                            1.19   
Under 18M                                            0.00   
All                                                100.00   

                                                          \
PRIM_CONTRIBUTORY_CAUSE DISREGARDING OTHER TRAFFIC SIGNS   
AGE_SEX_GROUP                                              
Between 19 and 30F                                  9.85   
Between 19 and 30M                                 16.67   
Middle AgeF                                        18.18   
Middle AgeM                                        39.39   
OlderF                                              3.79   
OlderM                                              9.09   
Under 18F                                           2.27   
Under 18M                                           0.76   
All                                               100.00   

                                                                           \
PRIM_CONTRIBUTORY_CAUSE DISREGARDING ROAD MARKINGS DISREGARDING STOP SIGN   
AGE_SEX_GROUP                                                               
Between 19 and 30F                            4.00                  13.93   
Between 19 and 30M                           12.00                  14.72   
Middle AgeF                                  18.67                  19.50   
Middle AgeM                                  34.67                  23.74   
OlderF                                        6.67                   7.82   
OlderM                                       22.67                  16.84   
Under 18F                                     0.00                   1.46   
Under 18M                                     1.33                   1.99   
All                                         100.00                 100.00   

                                                                              \
PRIM_CONTRIBUTORY_CAUSE DISREGARDING TRAFFIC SIGNALS DISREGARDING YIELD SIGN   
AGE_SEX_GROUP                                                                  

,Chi-square test,results
0,Pearson Chi-square ( 266.0) =,1131.5965
1,p-value =,0.0000
2,Cramer's V =,0.0552


In [40]:
table3.to_excel(r'/Users/shellylin/repo_homlsl/prim_chi_square.xlsx', index = False, header = True)

NotImplementedError: Writing to Excel with MultiIndex columns and no index ('index'=False) is not yet implemented.

In [ ]:
import researchpy as rp
table2, results2 = rp.crosstab(processed_data['AGE_SEX_GROUP'], processed_data['FIRST_CRASH_TYPE'], 
                             prop='col', test='chi-square')
    
display(table2)
results2
# 3*2
# SEX_AGED_GROUP vs. PRIM_CONTRIBUTORY

# Female and male at the same time
type(results2)

In [ ]:
trial19= pd.crosstab(index = [processed_data["INJURY"], processed_data["PHYSICAL_CONDITION"]], columns = [processed_data["AGE"], processed_data["SEX"]], margins=True)
trial19

In [ ]:
trial20= pd.crosstab(index = [processed_data["INJURY"], processed_data["CELL_PHONE_USE"]], columns = [processed_data["AGE"], processed_data["SEX"]], margins=True)
trial20

In [ ]:
trial21= pd.crosstab(index = [processed_data["INJURY"], processed_data["VEHICLE_AGE"]], columns = [processed_data["AGE"], processed_data["SEX"]], margins=True)
trial21

In [ ]:
processed_data.groupby(['BAC_RESULT VALUE']).size()

'1.1.3'